In [1]:
#setup Odoo

import requests

jsonrpcid = 0
url = "https://demo-28-08-6.adhoc.ar/jsonrpc"
db = "demo-28-08-6"
username = "Chargebee"
password = "Chargebee"
headers = {'Content-Type': 'application/json'}


class Odoo():

    def __init__(self):
        self.jsonrpcid = 0

    def call(self, method: str, service: str, *args):
        self.jsonrpcid += 1

        data = {
            "jsonrpc": "2.0",
            "method": "call",
            "params": {
                "service": service,
                "method": method,
                "args": args
            },
            "id": self.jsonrpcid
        }

        # Realizar solicitud HTTP POST
        response = requests.post(url, headers=headers, json=data)
        result = response.json()

        return result

    def login(self, db: str, username: str, password: str):
        self.db = db
        self.username = username
        self.password = password
        outcome = self.call('login', 'common', self.db, self.username, self.password)
        if 'result' not in outcome:
            raise Exception('Failed to login')
        self.uid = outcome['result']

    def execute(self, *args):
        result = self.call('execute_kw', 'object', self.db, self.uid, self.password, *args)
        r = result.get('result', False)
        if not r:
            print(result)
            raise Exception('Command has failed')
        return r

    def read(self, model: str, ids: list=[], fields: list=[], context: dict={}):
        return self.execute(model, 'read', [ids, fields], {'context': context})

    def search_read(self, model: str, domain: list=[], fields: list=[], offset: int=0, limit: int=10, context: dict={}):
        return self.execute(model, 'search_read', [domain, fields, offset, limit], {'context': context})

    def create(self, model: str, vals_list: list=[], context: dict={}):
        return self.execute(model, 'create', vals_list, {'context': context})

# Crear nueva instancia
odoo = Odoo()

# Realizar login
odoo.login(db, username, password)

In [2]:
#setup chargebee

import chargebee

chargebee.configure('live_3NEiAmTg7dQArgRHR04LvHaIABupcuQOS', 'cliengo')

In [93]:
# Obtener las invoices desde Chargebee

import pandas as pd
import time
def get_chargebee_invoices_august_2024():
    invoice_list = []
    offset = None  # Inicialmente sin offset para la paginación
    start_date = int(time.mktime(time.strptime("2024-08-01", "%Y-%m-%d")))  # 1 de agosto de 2024 en Unix Timestamp
    end_date = int(time.mktime(time.strptime("2024-08-31 23:59:59", "%Y-%m-%d %H:%M:%S")))  # 31 de agosto de 2024

    while True:
        # Obtener la lista de facturas, sin límite explícito, pero con paginación
        entries = chargebee.Invoice.list({
            "status[in]": ["paid", "payment_due"],
            "date[between]": [start_date, end_date],  # Filtro por rango de fechas en Unix Timestamp
            "sort_by[asc]": "date",
            "limit": 100,  # Procesar 100 facturas por página
            "offset": offset  # Usar el offset actual para paginación
        })

        # Agregar las facturas a la lista
        for entry in entries:
            invoice = entry.invoice
            invoice_list.append(invoice.__dict__)  # Agregar la factura a la lista

        # Verificar si hay más páginas (si existe un offset para la siguiente página)
        offset = entries.next_offset
        if not offset:
            break  # Salir del bucle si no hay más páginas

    return invoice_list

# Llamar la función para obtener todas las facturas de agosto de 2024
invoices = get_chargebee_invoices_august_2024()

In [94]:
# Convertir la lista de diccionarios en un DataFrame de pandas
dfchar = pd.DataFrame(invoices)

# Mostrar el DataFrame
dfchar

,values,sub_types,dependant_types,id,po_number,customer_id,subscription_id,recurring,status,vat_number,...,void_reason_code,deleted,tax_category,vat_number_prefix,channel,business_entity_id,site_details_at_creation,tax_origin,object,base_currency_code
0,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
1,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2,"{'id': '185394', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,True,paid,30025707,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
3,"{'id': '185395', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185395,Argentina - Transferencia Bancaria,5cb4d5d0e4b0bd6bed7869fa,16CY2RUEdBvEE1YCI,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
4,"{'id': '185396', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},185396,Chile,5b47de04e4b03c859495a0f7,Izy9TIBR04obyPV06,True,paid,969726408,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2442,"{'id': '187908', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187908,Argentina - Transferencia Bancaria (Leadaki),5f3da9f3b68252002aaac99e,AzZsu3UBeV4iY8HEC,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2443,"{'id': '187910', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187910,Argentina - Transferencia Bancaria (Leadaki),5a71ba83e4b07b95e31fc272,AzqBxKUBeW38u8EGG,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2444,"{'id': '187892', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187892,Argentina - Transferencia Bancaria (Leadaki),57bddbafe4b08f5c877fe039,169v0eUBcr4Bg73Qq,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2445,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD


In [95]:
df = dfchar.rename(columns={'id': 'invoice_id'})
df

,values,sub_types,dependant_types,invoice_id,po_number,customer_id,subscription_id,recurring,status,vat_number,...,void_reason_code,deleted,tax_category,vat_number_prefix,channel,business_entity_id,site_details_at_creation,tax_origin,object,base_currency_code
0,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
1,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2,"{'id': '185394', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,True,paid,30025707,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
3,"{'id': '185395', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185395,Argentina - Transferencia Bancaria,5cb4d5d0e4b0bd6bed7869fa,16CY2RUEdBvEE1YCI,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
4,"{'id': '185396', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},185396,Chile,5b47de04e4b03c859495a0f7,Izy9TIBR04obyPV06,True,paid,969726408,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2442,"{'id': '187908', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187908,Argentina - Transferencia Bancaria (Leadaki),5f3da9f3b68252002aaac99e,AzZsu3UBeV4iY8HEC,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2443,"{'id': '187910', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187910,Argentina - Transferencia Bancaria (Leadaki),5a71ba83e4b07b95e31fc272,AzqBxKUBeW38u8EGG,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2444,"{'id': '187892', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187892,Argentina - Transferencia Bancaria (Leadaki),57bddbafe4b08f5c877fe039,169v0eUBcr4Bg73Qq,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2445,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD


In [96]:
#para separar los campos dentro de jsons y diccionarios genere el siguiente codigo @Yepes
def print_column_types(dataframe):
    for column in dataframe.columns:
        dtype = dataframe[column].dtype
        print(f"{column} \t {dtype}")

# Llamar a la función para mostrar las columnas y tipos de datos
print_column_types(df)

values 	 object
sub_types 	 object
dependant_types 	 object
invoice_id 	 object
po_number 	 object
customer_id 	 object
subscription_id 	 object
recurring 	 bool
status 	 object
vat_number 	 object
price_type 	 object
date 	 int64
due_date 	 int64
net_term_days 	 int64
exchange_rate 	 float64
currency_code 	 object
total 	 int64
amount_paid 	 int64
amount_adjusted 	 int64
write_off_amount 	 int64
credits_applied 	 int64
amount_due 	 int64
paid_at 	 int64
dunning_status 	 object
next_retry_at 	 object
voided_at 	 object
resource_version 	 int64
updated_at 	 int64
sub_total 	 int64
sub_total_in_local_currency 	 object
total_in_local_currency 	 object
local_currency_code 	 object
tax 	 int64
local_currency_exchange_rate 	 object
first_invoice 	 bool
new_sales_amount 	 float64
has_advance_charges 	 bool
term_finalized 	 bool
is_gifted 	 bool
generated_at 	 int64
expected_payment_date 	 float64
amount_to_collect 	 int64
round_off_amount 	 int64
line_items 	 object
discounts 	 object
line_it

In [97]:
df_objects = df.select_dtypes(include=['object'])
df_objects

,values,sub_types,dependant_types,invoice_id,po_number,customer_id,subscription_id,status,vat_number,price_type,...,payment_owner,void_reason_code,tax_category,vat_number_prefix,channel,business_entity_id,site_details_at_creation,tax_origin,object,base_currency_code
0,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,paid,None,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
1,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,paid,None,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2,"{'id': '185394', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,paid,30025707,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
3,"{'id': '185395', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185395,Argentina - Transferencia Bancaria,5cb4d5d0e4b0bd6bed7869fa,16CY2RUEdBvEE1YCI,paid,None,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
4,"{'id': '185396', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},185396,Chile,5b47de04e4b03c859495a0f7,Izy9TIBR04obyPV06,paid,969726408,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2442,"{'id': '187908', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187908,Argentina - Transferencia Bancaria (Leadaki),5f3da9f3b68252002aaac99e,AzZsu3UBeV4iY8HEC,paid,None,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2443,"{'id': '187910', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187910,Argentina - Transferencia Bancaria (Leadaki),5a71ba83e4b07b95e31fc272,AzqBxKUBeW38u8EGG,paid,None,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2444,"{'id': '187892', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187892,Argentina - Transferencia Bancaria (Leadaki),57bddbafe4b08f5c877fe039,169v0eUBcr4Bg73Qq,paid,None,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2445,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,paid,None,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD


In [23]:
# List of columns to un-nest
#columnas_a_desanidar = ['values', 'sub_types']

     dependant_types invoice_id                                     po_number  \
0                 {}     185392            Argentina - Transferencia Bancaria   
1                 {}     185393            Argentina - Transferencia Bancaria   
2                 {}     185394            Argentina - Transferencia Bancaria   
3                 {}     185395            Argentina - Transferencia Bancaria   
4                 {}     185396                                         Chile   
...              ...        ...                                           ...   
2441              {}     187908  Argentina - Transferencia Bancaria (Leadaki)   
2442              {}     187910  Argentina - Transferencia Bancaria (Leadaki)   
2443              {}     187892  Argentina - Transferencia Bancaria (Leadaki)   
2444              {}     187893  Argentina - Transferencia Bancaria (Leadaki)   
2445              {}     187914                                         Chile   

                   customer

In [98]:
#y lo que aplico a este otro dataframe es la logica de separacion para json, que estos estaban raros por ese \n
dict_df2 = df_objects[['dependant_types', 'line_items', 'discounts', 'line_item_tiers', 'linked_payments',
'dunning_attempts', 'applied_credits', 'adjustment_credit_notes', 'issued_credit_notes', 'linked_orders', 'billing_address',
'site_details_at_creation', 'tax_origin']]
dict_df2

,dependant_types,line_items,discounts,line_item_tiers,linked_payments,dunning_attempts,applied_credits,adjustment_credit_notes,issued_credit_notes,linked_orders,billing_address,site_details_at_creation,tax_origin
0,{},"[{\n ""id"": ""li_6olL2UKBV7cg22wd"",\n ""dat...",None,"[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_AzqB1XUMIWXep30Ov"",\n ...","[{\n ""created_at"": 1722567797,\n ""attemp...",[],[],[],[],"{\n ""company"": ""PARRA E HIJOS S.A."",\n ""...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
1,{},"[{\n ""id"": ""li_Azz6aoUKC1bN11M2p"",\n ""da...","[{\n ""object"": ""discount"",\n ""entity_typ...","[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_16CXuLULLoKeC2h8c"",\n ...","[{\n ""created_at"": 1722568032,\n ""attemp...",[],[],[],[],"{\n ""company"": ""Nuke"",\n ""country"": ""AR""...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
2,{},"[{\n ""id"": ""li_AzyjxKUKC78gO1LOQ"",\n ""da...",None,"[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_16BW6dUJLpJrc1Cpj"",\n ...",[],[],[],[],[],"{\n ""first_name"": ""Martin"",\n ""last_name...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
3,{},"[{\n ""id"": ""li_6olL2UKCAuip39NE"",\n ""dat...",None,"[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_AzqBCbUKy7RvD2RIN"",\n ...","[{\n ""created_at"": 1722567951,\n ""attemp...",[],[],[],[],"{\n ""company"": ""http://www.perkinelmer.com....","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
4,{},"[{\n ""id"": ""li_6olL2UKCDiUk3E1u"",\n ""dat...",None,"[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_6olL2UKCDiWS3E1z"",\n ...",[],[],[],[],[],"{\n ""first_name"": ""Francisco"",\n ""last_n...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2442,{},"[{\n ""id"": ""li_16BdL2UN5VwgXKJF5"",\n ""da...","[{\n ""object"": ""discount"",\n ""entity_typ...","[{\n ""starting_unit"": 1,\n ""ending_unit""...",[],[],[],[],[],[],"{\n ""company"": ""Renault Directa"",\n ""cou...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
2443,{},"[{\n ""id"": ""li_16BdL2UN5Vwo1KJFY"",\n ""da...","[{\n ""object"": ""discount"",\n ""entity_typ...","[{\n ""starting_unit"": 1,\n ""ending_unit""...",[],[],[],[],[],[],"{\n ""company"": ""tuprode.com"",\n ""country...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
2444,{},"[{\n ""id"": ""li_16BdL2UN5VvwOKJC3"",\n ""da...","[{\n ""object"": ""discount"",\n ""entity_typ...","[{\n ""starting_unit"": 1,\n ""ending_unit""...",[],[],[],[],"[{\n ""cn_id"": ""CN-7066"",\n ""cn_reason_co...",[],"{\n ""first_name"": ""Mica"",\n ""last_name"":...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
2445,{},"[{\n ""id"": ""li_16BdL2UN5VvzbKJCO"",\n ""da...","[{\n ""object"": ""discount"",\n ""entity_typ...","[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_16CXx1UO5eUVR3dvn"",\n ...","[{\n ""created_at"": 1725246813,\n ""attemp...",[],[],[],[],"{\n ""first_name"": ""Lucas"",\n ""last_name""...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"


In [78]:
#df[['invoice_id', 'line_items']]

,dependant_types,line_items,discounts,line_item_tiers,linked_payments,dunning_attempts,applied_credits,adjustment_credit_notes,issued_credit_notes,linked_orders,billing_address,site_details_at_creation,tax_origin


In [100]:
import pandas as pd
import json

# Step 1: Extract line_items list from the nested structure in 'values' if it's JSON-like
# Check if 'line_items' is stored within another dictionary (e.g., within 'values' column)
df['line_items'] = df['values'].apply(lambda x: json.loads(x).get('line_items') if isinstance(x, str) else x.get('line_items') if isinstance(x, dict) else None)

# Step 2: Explode line_items into individual rows
df_exploded = df.explode('line_items', ignore_index=True)

# Step 3: Normalize the line_items dictionaries into columns
line_items_df = pd.json_normalize(df_exploded['line_items'])

# Step 4: Drop the 'line_items' column from the original exploded DataFrame
df_exploded = df_exploded.drop(columns=['line_items'])

# Step 5: Concatenate the main DataFrame with the expanded line items DataFrame
df_final = pd.concat([df_exploded, line_items_df], axis=1)

# Display the final DataFrame
df_final

,values,sub_types,dependant_types,invoice_id,po_number,customer_id,subscription_id,recurring,status,vat_number,...,object,subscription_id,customer_id,description,entity_type,entity_id,entity_description,tax_exempt_reason,discount_amount,item_level_discount_amount
0,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,None,...,line_item,169uzAU4TrwKK33PY,5acdf7d8e4b014cba3b1724b,Cliengo Premium,plan,CLIENGO_PREMIUM_CONVER,"Cliengo Premium, 1000 conversaciones al mes",tax_not_configured,0,0
1,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,None,...,line_item,169uzAU4TrwKK33PY,5acdf7d8e4b014cba3b1724b,Agentes,addon,CLIENGO_PREMIUM_CONVER_USERS,NaN,tax_not_configured,0,0
2,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,line_item,6or0jU5PxxMNEhuk,5aa291c2e4b0c65fcc798ce8,Cliengo Premium,plan,CLIENGO_PREMIUM_CONVER,"Cliengo Premium, 1000 conversaciones al mes",tax_not_configured,0,0
3,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,line_item,6or0jU5PxxMNEhuk,5aa291c2e4b0c65fcc798ce8,Agentes,addon,CLIENGO_PREMIUM_CONVER_USERS,NaN,tax_not_configured,2000,2000
4,"{'id': '185394', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,True,paid,30025707,...,line_item,AzZN3tT4OBaWmiMY,5ad08c66e4b01bf0eefaf887,Cliengo Starter,plan,CLIENGO_STARTER_50_V2,NaN,tax_not_configured,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5702,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,None,...,line_item,16CVC3UBe8bt98ESR,5d3773fae4b0c496f93c2467,Agentes,addon,CLIENGO_PREMIUM_CONVER_USERS,NaN,tax_not_configured,6000,6000
5703,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,None,...,line_item,16CVC3UBe8bt98ESR,5d3773fae4b0c496f93c2467,Whatsapp Premium Included,addon,WHATSAPP_PREMIUM_INCLUDED,NaN,tax_not_configured,0,0
5704,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,None,...,line_item,16CVC3UBe8bt98ESR,5d3773fae4b0c496f93c2467,Cliengo IA Premium,addon,cliengo-ia-premium,NaN,tax_not_configured,0,0
5705,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,5a0c58c9e4b031a2c96d319c,1mkVvlDQwVxVgL8H4,True,paid,76.159.278-5,...,line_item,1mkVvlDQwVxVgL8H4,5a0c58c9e4b031a2c96d319c,Cliengo Starter,plan,CLIENGO_STARTER_CONVER,"Cliengo Starter, 250 conversaciones al mes",tax_not_configured,0,0


In [101]:
# Step 1: Extract discounts list from the nested structure in 'df_final'
df_final['discounts'] = df_final['values'].apply(lambda x: x.get('discounts') if isinstance(x, dict) else None)

# Step 2: Handle duplicate column names by renaming them
def rename_duplicates(df):
    cols = pd.Series(df.columns)
    for dup in df.columns[df.columns.duplicated()].unique():
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
    df.columns = cols
    return df

# Rename duplicate columns in df_final before exploding
df_final = rename_duplicates(df_final)

# Step 3: Explode discounts into individual rows
df_final_exploded = df_final.explode('discounts', ignore_index=True)

# Step 4: Normalize the discounts dictionaries into columns and add a prefix to avoid duplicates
discounts_df = pd.json_normalize(df_final_exploded['discounts']).add_prefix('discounts_')

# Step 5: Drop the original 'discounts' column from the exploded DataFrame
df_final_exploded = df_final_exploded.drop(columns=['discounts'])
df_final_exploded = df_final_exploded.drop(columns=['line_item_discounts'])

# Step 6: Concatenate the main DataFrame with the expanded discounts DataFrame
df_discounts = pd.concat([df_final_exploded, discounts_df], axis=1)

# Display the final DataFrame with renamed discounts columns
df_discounts


,values,sub_types,dependant_types,invoice_id,po_number,customer_id,subscription_id,recurring,status,vat_number,...,tax_exempt_reason,discount_amount,item_level_discount_amount,discounts_object,discounts_entity_type,discounts_description,discounts_amount,discounts_entity_id,discounts_discount_type,discounts_discount_percentage
0,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,None,...,tax_not_configured,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,None,...,tax_not_configured,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,tax_not_configured,0,0,discount,item_level_coupon,Bonificación usuarios,2000.0,Migra2024Premium2Users,fixed_amount,NaN
3,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,tax_not_configured,2000,2000,discount,item_level_coupon,Bonificación usuarios,2000.0,Migra2024Premium2Users,fixed_amount,NaN
4,"{'id': '185394', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,True,paid,30025707,...,tax_not_configured,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5725,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,None,...,tax_not_configured,6000,6000,discount,item_level_coupon,100%PLAN,18900.0,LEADAKIIA100%PLAN,percentage,100.0
5726,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,None,...,tax_not_configured,0,0,discount,item_level_coupon,100%PLAN,18900.0,LEADAKIIA100%PLAN,percentage,100.0
5727,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,None,...,tax_not_configured,0,0,discount,item_level_coupon,100%PLAN,18900.0,LEADAKIIA100%PLAN,percentage,100.0
5728,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,5a0c58c9e4b031a2c96d319c,1mkVvlDQwVxVgL8H4,True,paid,76.159.278-5,...,tax_not_configured,0,0,discount,item_level_coupon,Bonificación usuarios,2100.0,MIGRA2024STARTER3USERS,fixed_amount,NaN


In [102]:
# Step 1: Extract discounts list from the nested structure in 'df_final'
df_discounts['linked_payments'] = df_discounts['values'].apply(lambda x: x.get('linked_payments') if isinstance(x, dict) else None)


# Rename duplicate columns in df_final before exploding
df_discounts = rename_duplicates(df_discounts)

# Step 3: Explode discounts into individual rows
df_discounts_exploded = df_discounts.explode('linked_payments', ignore_index=True)

# Step 4: Normalize the discounts dictionaries into columns and add a prefix to avoid duplicates
payments_df = pd.json_normalize(df_discounts_exploded['linked_payments']).add_prefix('linked_payments_')

# Step 5: Drop the original 'discounts' column from the exploded DataFrame
df_discounts_exploded = df_discounts_exploded.drop(columns=['linked_payments'])

# Step 6: Concatenate the main DataFrame with the expanded discounts DataFrame
df_payments = pd.concat([df_discounts_exploded, payments_df], axis=1)

# Display the final DataFrame with renamed discounts columns
df_payments

,values,sub_types,dependant_types,invoice_id,po_number,customer_id,subscription_id,recurring,status,vat_number,...,discounts_amount,discounts_entity_id,discounts_discount_type,discounts_discount_percentage,linked_payments_txn_id,linked_payments_applied_amount,linked_payments_applied_at,linked_payments_txn_status,linked_payments_txn_date,linked_payments_txn_amount
0,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,None,...,NaN,NaN,NaN,NaN,txn_AzqB1XUMIWXep30Ov,8900.0,1.724436e+09,success,1.724349e+09,8900.0
1,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,None,...,NaN,NaN,NaN,NaN,txn_AzqB1XUMIWXep30Ov,8900.0,1.724436e+09,success,1.724349e+09,8900.0
2,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,2000.0,Migra2024Premium2Users,fixed_amount,NaN,txn_16CXuLULLoKeC2h8c,9900.0,1.723568e+09,success,1.723050e+09,9900.0
3,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,2000.0,Migra2024Premium2Users,fixed_amount,NaN,txn_16CXuLULLoKeC2h8c,9900.0,1.723568e+09,success,1.723050e+09,9900.0
4,"{'id': '185394', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,True,paid,30025707,...,NaN,NaN,NaN,NaN,txn_16BW6dUJLpJrc1Cpj,3900.0,1.722509e+09,success,1.721012e+09,11700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7880,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,None,...,18900.0,LEADAKIIA100%PLAN,percentage,100.0,txn_16CXx1UO5eUVR3dvn,5000.0,1.726078e+09,success,1.725991e+09,5000.0
7881,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,None,...,18900.0,LEADAKIIA100%PLAN,percentage,100.0,txn_16CXx1UO5eUVR3dvn,5000.0,1.726078e+09,success,1.725991e+09,5000.0
7882,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,None,...,18900.0,LEADAKIIA100%PLAN,percentage,100.0,txn_16CXx1UO5eUVR3dvn,5000.0,1.726078e+09,success,1.725991e+09,5000.0
7883,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,5a0c58c9e4b031a2c96d319c,1mkVvlDQwVxVgL8H4,True,paid,76.159.278-5,...,2100.0,MIGRA2024STARTER3USERS,fixed_amount,NaN,txn_16BdL2UN5XW31KMPH,3900.0,1.725160e+09,success,1.725160e+09,3900.0


In [103]:
# Step 1: Extract 'billing_address' from the nested structure in 'df_payments'
df_payments['billing_address'] = df_payments['values'].apply(lambda x: x.get('billing_address') if isinstance(x, dict) else None)

# Step 2: Normalize the 'billing_address' dictionaries into columns and add a prefix to avoid duplicates
billing_address_df = pd.json_normalize(df_payments['billing_address']).add_prefix('billing_address_')

# Step 3: Drop the original 'billing_address' column from df_payments
df_payments = df_payments.drop(columns=['billing_address'])

# Step 4: Concatenate the main DataFrame with the expanded 'billing_address' DataFrame
df_payments = pd.concat([df_payments, billing_address_df], axis=1)

# Display the final DataFrame with the expanded billing address columns
df_payments

,values,sub_types,dependant_types,invoice_id,po_number,customer_id,subscription_id,recurring,status,vat_number,...,billing_address_last_name,billing_address_city,billing_address_state,billing_address_zip,billing_address_line1,billing_address_email,billing_address_state_code,billing_address_line2,billing_address_phone,billing_address_line3
0,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'id': '185394', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,True,paid,30025707,...,Golszmidt,CABA,CABA,1424,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7880,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,None,...,Raz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7881,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,None,...,Raz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7882,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,None,...,Raz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7883,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,5a0c58c9e4b031a2c96d319c,1mkVvlDQwVxVgL8H4,True,paid,76.159.278-5,...,Rosales,Santiago,R.Metropolitana,NaN,Santa Corina 097,NaN,NaN,NaN,NaN,NaN


In [104]:
# Step 1: Extract 'billing_address' from the nested structure in 'df_payments'
df_payments['shipping_address'] = df_payments['values'].apply(lambda x: x.get('shipping_address') if isinstance(x, dict) else None)

# Step 2: Normalize the 'billing_address' dictionaries into columns and add a prefix to avoid duplicates
billing_address_df = pd.json_normalize(df_payments['shipping_address']).add_prefix('shipping_address_')

# Step 3: Drop the original 'billing_address' column from df_payments
df_payments = df_payments.drop(columns=['shipping_address'])

# Step 4: Concatenate the main DataFrame with the expanded 'billing_address' DataFrame
df_payments = pd.concat([df_payments, billing_address_df], axis=1)

# Display the final DataFrame with the expanded billing address columns
df_payments

,values,sub_types,dependant_types,invoice_id,po_number,customer_id,subscription_id,recurring,status,vat_number,...,shipping_address_first_name,shipping_address_company,shipping_address_last_name,shipping_address_city,shipping_address_email,shipping_address_line1,shipping_address_state,shipping_address_zip,shipping_address_phone,shipping_address_line2
0,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'id': '185394', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,True,paid,30025707,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7880,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7881,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7882,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7883,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,5a0c58c9e4b031a2c96d319c,1mkVvlDQwVxVgL8H4,True,paid,76.159.278-5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
# Step 1: Extract 'line_item_tiers' from the nested structure in 'df_payments'
df_payments['line_item_tiers'] = df_payments['values'].apply(lambda x: x.get('line_item_tiers') if isinstance(x, dict) else None)

# Step 2: Explode 'line_item_tiers' into individual rows if it's a list of dictionaries
df_payments_exploded = df_payments.explode('line_item_tiers', ignore_index=True)

# Step 3: Normalize the 'line_item_tiers' dictionaries into columns and add a prefix to avoid duplicates
line_item_tiers_df = pd.json_normalize(df_payments_exploded['line_item_tiers']).add_prefix('line_item_tiers_')

# Step 4: Drop the original 'line_item_tiers' column from the exploded DataFrame
df_payments_exploded = df_payments_exploded.drop(columns=['line_item_tiers'])

# Step 5: Concatenate the main DataFrame with the expanded 'line_item_tiers' DataFrame
df_payments = pd.concat([df_payments_exploded, line_item_tiers_df], axis=1)

# Display the final DataFrame with the expanded line_item_tiers columns
df_payments

,values,sub_types,dependant_types,invoice_id,po_number,customer_id,subscription_id,recurring,status,vat_number,...,shipping_address_state,shipping_address_zip,shipping_address_phone,shipping_address_line2,line_item_tiers_starting_unit,line_item_tiers_ending_unit,line_item_tiers_quantity_used,line_item_tiers_unit_amount,line_item_tiers_object,line_item_tiers_line_item_id
0,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,None,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,line_item_tier,li_6olL2UKBV7cm22we
1,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,None,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,line_item_tier,li_6olL2UKBV7cm22we
2,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,line_item_tier,li_Azz6aoUKC1bN81M2q
3,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,NaN,NaN,NaN,NaN,2.0,NaN,3.0,1000.0,line_item_tier,li_Azz6aoUKC1bN81M2q
4,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,line_item_tier,li_Azz6aoUKC1bN81M2q
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10394,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,None,...,NaN,NaN,NaN,NaN,2.0,NaN,6.0,1000.0,line_item_tier,li_16BdL2UN5VvziKJCQ
10395,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,5a0c58c9e4b031a2c96d319c,1mkVvlDQwVxVgL8H4,True,paid,76.159.278-5,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,line_item_tier,li_16BdL2UN5XW1GKMPE
10396,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,5a0c58c9e4b031a2c96d319c,1mkVvlDQwVxVgL8H4,True,paid,76.159.278-5,...,NaN,NaN,NaN,NaN,2.0,NaN,3.0,700.0,line_item_tier,li_16BdL2UN5XW1GKMPE
10397,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,5a0c58c9e4b031a2c96d319c,1mkVvlDQwVxVgL8H4,True,paid,76.159.278-5,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,line_item_tier,li_16BdL2UN5XW1GKMPE


In [106]:
# Step 1: Extract 'line_item_tiers' from the nested structure in 'df_payments'
df_payments['issued_credit_notes'] = df_payments['values'].apply(lambda x: x.get('issued_credit_notes') if isinstance(x, dict) else None)

# Step 2: Explode 'line_item_tiers' into individual rows if it's a list of dictionaries
df_payments_exploded = df_payments.explode('issued_credit_notes', ignore_index=True)

# Step 3: Normalize the 'line_item_tiers' dictionaries into columns and add a prefix to avoid duplicates
line_item_tiers_df = pd.json_normalize(df_payments_exploded['issued_credit_notes']).add_prefix('issued_credit_notes_')

# Step 4: Drop the original 'line_item_tiers' column from the exploded DataFrame
df_payments_exploded = df_payments_exploded.drop(columns=['issued_credit_notes'])

# Step 5: Concatenate the main DataFrame with the expanded 'line_item_tiers' DataFrame
df_payments = pd.concat([df_payments_exploded, line_item_tiers_df], axis=1)

# Display the final DataFrame with the expanded line_item_tiers columns
df_payments

,values,sub_types,dependant_types,invoice_id,po_number,customer_id,subscription_id,recurring,status,vat_number,...,line_item_tiers_quantity_used,line_item_tiers_unit_amount,line_item_tiers_object,line_item_tiers_line_item_id,issued_credit_notes_cn_id,issued_credit_notes_cn_reason_code,issued_credit_notes_cn_create_reason_code,issued_credit_notes_cn_date,issued_credit_notes_cn_total,issued_credit_notes_cn_status
0,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,None,...,1.0,0.0,line_item_tier,li_6olL2UKBV7cm22we,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,None,...,1.0,0.0,line_item_tier,li_6olL2UKBV7cm22we,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,1.0,0.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,3.0,1000.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,1.0,0.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10474,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,None,...,6.0,1000.0,line_item_tier,li_16BdL2UN5VvziKJCQ,NaN,NaN,NaN,NaN,NaN,NaN
10475,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,5a0c58c9e4b031a2c96d319c,1mkVvlDQwVxVgL8H4,True,paid,76.159.278-5,...,1.0,0.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
10476,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,5a0c58c9e4b031a2c96d319c,1mkVvlDQwVxVgL8H4,True,paid,76.159.278-5,...,3.0,700.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
10477,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,5a0c58c9e4b031a2c96d319c,1mkVvlDQwVxVgL8H4,True,paid,76.159.278-5,...,1.0,0.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
#Le cambie la region a argentina utc tal y como lo tuvimos que hacer con panella cuando los traiamos de BigQuery antes
import pytz
argentina_tz = pytz.timezone('America/Argentina/Buenos_Aires')
df_payments['date_from'] = pd.to_datetime(df_payments['date_from'], unit='s', utc=True).dt.tz_convert(argentina_tz).dt.date
df_payments['date_to'] = pd.to_datetime(df_payments['date_to'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_payments['generated_at'] = pd.to_datetime(df_payments['generated_at'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_payments['updated_at'] = pd.to_datetime(df_payments['updated_at'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_payments['paid_at'] = pd.to_datetime(df_payments['paid_at'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_payments['due_date'] = pd.to_datetime(df_payments['due_date'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_payments['date'] = pd.to_datetime(df_payments['date'], unit='s', utc=True).dt.tz_convert(argentina_tz)

In [110]:
order = df_payments.sort_values(by='invoice_id', ascending=False)
order

,values,sub_types,dependant_types,invoice_id,po_number,customer_id,subscription_id,recurring,status,vat_number,...,line_item_tiers_quantity_used,line_item_tiers_unit_amount,line_item_tiers_object,line_item_tiers_line_item_id,issued_credit_notes_cn_id,issued_credit_notes_cn_reason_code,issued_credit_notes_cn_create_reason_code,issued_credit_notes_cn_date,issued_credit_notes_cn_total,issued_credit_notes_cn_status
10478,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,5a0c58c9e4b031a2c96d319c,1mkVvlDQwVxVgL8H4,True,paid,76.159.278-5,...,3.0,700.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
10477,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,5a0c58c9e4b031a2c96d319c,1mkVvlDQwVxVgL8H4,True,paid,76.159.278-5,...,1.0,0.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
10476,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,5a0c58c9e4b031a2c96d319c,1mkVvlDQwVxVgL8H4,True,paid,76.159.278-5,...,3.0,700.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
10475,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,5a0c58c9e4b031a2c96d319c,1mkVvlDQwVxVgL8H4,True,paid,76.159.278-5,...,1.0,0.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
9723,"{'id': '187913', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187913,Argentina - Transferencia Bancaria (Leadaki),536109b2e4b0ae650c483374,16CY4nUBd1nGm7OLS,True,paid,None,...,5.0,0.0,line_item_tier,li_Azyk2WUN5VwXQ7GuX,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,1.0,0.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
5,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,3.0,1000.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,3.0,1000.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,None,...,1.0,0.0,line_item_tier,li_6olL2UKBV7cm22we,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
#print_column_types(order)

id 	 object
po_number 	 object
customer_id 	 object
subscription_id 	 object
recurring 	 bool
status 	 object
price_type 	 object
date 	 datetime64[ns, America/Argentina/Buenos_Aires]
due_date 	 datetime64[ns, America/Argentina/Buenos_Aires]
net_term_days 	 int64
exchange_rate 	 float64
total 	 int64
amount_paid 	 int64
amount_adjusted 	 int64
write_off_amount 	 int64
credits_applied 	 int64
amount_due 	 int64
paid_at 	 datetime64[ns, America/Argentina/Buenos_Aires]
dunning_status 	 object
updated_at 	 datetime64[ns, America/Argentina/Buenos_Aires]
resource_version 	 int64
deleted 	 bool
object 	 object
first_invoice 	 bool
amount_to_collect 	 int64
round_off_amount 	 int64
has_advance_charges 	 bool
currency_code 	 object
base_currency_code 	 object
generated_at 	 datetime64[ns, America/Argentina/Buenos_Aires]
is_gifted 	 bool
term_finalized 	 bool
channel 	 object
tax 	 int64
sub_total 	 int64
linked_orders 	 object
billing_address.company 	 object
billing_address.country 	 object
bi

In [ ]:
#a partir de aqui empieza lo de Odoo

In [17]:
#primero con las pruebas de insercion en sales.order


odoo.create('sale.order', [{
                            'campaign_id': False,
                            'source_id': False,
                            'medium_id': False,
                            'activity_ids': [],
                            'activity_state': True,
                            'partner_id': 11, 
                            'company_id': 1,
                            'website_id': 1,
                            }])


356

In [16]:
# aqui yo buscaba antes un id recien creado por el codigo anterior
sale_order_345 = odoo.execute('sale.order', 'search_read', [[['id', '=', 354]], [], 0, 1])
print(sale_order_345)

[{'id': 354, 'campaign_id': False, 'source_id': False, 'medium_id': False, 'activity_ids': [], 'activity_state': False, 'activity_user_id': False, 'activity_type_id': False, 'activity_type_icon': False, 'activity_date_deadline': False, 'my_activity_date_deadline': False, 'activity_summary': False, 'activity_exception_decoration': False, 'activity_exception_icon': False, 'activity_calendar_event_id': False, 'message_is_follower': True, 'message_follower_ids': [1243], 'message_partner_ids': [121], 'message_ids': [2745], 'has_message': True, 'message_needaction': False, 'message_needaction_counter': 0, 'message_has_error': False, 'message_has_error_counter': 0, 'message_attachment_count': 0, 'rating_ids': [], 'website_message_ids': [], 'message_has_sms_error': False, 'access_url': '/my/orders/354', 'access_token': False, 'access_warning': '', 'name': 'V 0001-00000342', 'company_id': [1, 'REGEX LLC'], 'partner_id': [11, 'Gemini Furniture'], 'state': 'draft', 'locked': False, 'client_order_

In [16]:
#aqui hacemos el llamado a las invoices de clientes en Odoo

#odf = odoo.execute('account.move', 'search_read', [[['id', '>', 1]], 0, 20])
#dfc = pd.DataFrame(odf)
#dfc

In [17]:
#este nada mas es una busqueda de una factura en especifico por medio del campo name, 
# que es el que aparece en la interfaz grafica de odoo


#dfc.loc[dfc['name'] == 'BNK1/2024/00007']

In [18]:
#de la funcion que cree al principio se la aplique al dataframe de las invoices de odoo 
# para ver el tipo de datos de cada campo del modelo account.move


#print_column_types(dfc)

In [19]:
#aqui hice el llamado de los campos de tipo float del dataframe para ver los campos que ocupan el equivalente a un valor de moneda
#lo hice para ir viendo los campos que mas adelante debo comparar con los de Chargebee y hacer la migracion correctamente


#floats = dfc.select_dtypes(include=['float64'])
#floats

In [20]:
#este codigo me muestra en un dataframe solamente los campos de tipo float de account.move pero sin excluir el id de la misma
# por eso le hago un concat con el original para que conserve el id

#if 'id' in dfc.columns:
#    floats_with_id = pd.concat([floats, dfc[['id', 'name']]], axis=1)
#else:
#    print("La columna 'id' no existe en el DataFrame original.")

#floats_with_id

In [21]:
#aqui fue en donde vi como es que se almacena el registro de partner_id (que es un array)

#sub_dfc = dfc['partner_id']
#sub_dfc

In [111]:
# Leer todas las facturas (sin establecer un dominio específico) todo para comprobar que la insercion fue exitosa
invoices = odoo.search_read('account.move', [], ['id', 'name', 'date', 'amount_total', 'partner_id'])

customers_invoice = pd.DataFrame(invoices)
customers_invoice


,id,name,date,amount_total,partner_id
0,164,/,2024-09-20,28.75,"[11, Gemini Furniture]"
1,161,INV/2024/00011,2024-09-17,28.75,"[11, Gemini Furniture]"
2,163,/,2024-09-17,0.00,False
3,160,/,2024-09-17,0.00,"[11, Gemini Furniture]"
4,159,/,2024-09-16,28.75,"[11, Gemini Furniture]"
5,158,/,2024-09-16,28.75,"[11, Gemini Furniture]"
6,157,/,2024-09-16,0.00,"[11, Gemini Furniture]"
7,162,INV/2024/00012,2024-09-10,575.00,"[10, Deco Addict]"
8,50,/,2024-08-31,0.00,False
9,156,INV/2024/00010,2024-08-28,28.75,"[11, Gemini Furniture]"


In [112]:
#llame a todas las invoice otra vez pero para separar los campos del array que muestra el partner_id 
# (al final no sirvio porque solamente fue una operacion  de lectura :(  )
i = odoo.search_read('account.move', [], [], limit=None)
inv = pd.DataFrame(i)
inv

#

,id,campaign_id,source_id,medium_id,is_tax_computed_externally,is_avatax,sequence_prefix,sequence_number,activity_ids,activity_state,...,l10n_ar_afip_asoc_period_start,l10n_ar_afip_asoc_period_end,l10n_ar_boarding_permission_ids,timesheet_ids,timesheet_count,timesheet_encode_uom_id,timesheet_total_duration,document_type_internal_type,invoice_validation_type,l10n_ar_edi_warning
0,164,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
1,161,False,False,False,False,False,INV/2024/,11,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2,163,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
3,160,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
4,159,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2493,19,False,False,False,False,False,BNK1/2024/,3,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2494,18,False,False,False,False,False,BNK1/2024/,2,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2495,17,False,False,False,False,False,BNK1/2024/,1,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2496,11,False,False,False,False,False,RBILL/2023/05/,1,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False


In [113]:
#solo mostrar el campo de partner_id para ver como quedan los customer_id de Chargebee  en Odoo
inv[['id', 'partner_id', 'name', 'invoice_date', 'invoice_line_ids']]

,id,partner_id,name,invoice_date,invoice_line_ids
0,164,"[11, Gemini Furniture]",/,False,[]
1,161,"[11, Gemini Furniture]",INV/2024/00011,2024-09-17,[473]
2,163,False,/,False,[479]
3,160,"[11, Gemini Furniture]",/,False,[]
4,159,"[11, Gemini Furniture]",/,False,[]
...,...,...,...,...,...
2493,19,False,BNK1/2024/00003,False,"[57, 58]"
2494,18,"[14, Azure Interior]",BNK1/2024/00002,False,"[55, 56]"
2495,17,False,BNK1/2024/00001,False,"[53, 54]"
2496,11,"[14, Azure Interior]",RBILL/2023/05/0001,2023-05-19,[20]


In [114]:
#realice el llamado del campo del customer de Chargebee para ingresarlo en res.partner
customerdf = order['customer_id']
customerdf

10478    5a0c58c9e4b031a2c96d319c
10477    5a0c58c9e4b031a2c96d319c
10476    5a0c58c9e4b031a2c96d319c
10475    5a0c58c9e4b031a2c96d319c
9723     536109b2e4b0ae650c483374
                   ...           
2        5aa291c2e4b0c65fcc798ce8
5        5aa291c2e4b0c65fcc798ce8
3        5aa291c2e4b0c65fcc798ce8
1        5acdf7d8e4b014cba3b1724b
0        5acdf7d8e4b014cba3b1724b
Name: customer_id, Length: 10479, dtype: object

In [115]:
#como los customers que vamos a ingresar estaban duplicados en el llamado, pues elimine los duplicados ya que a partir de este dataframe 
#hare las inserciones en el modelo res.partner
customers = customerdf.drop_duplicates()
customers

10478    5a0c58c9e4b031a2c96d319c
9723     536109b2e4b0ae650c483374
10448    53c42a63e4b08c5d83b878ff
10439    5727680de4b06a6e4222f0c8
10459    5a71ba83e4b07b95e31fc272
                   ...           
12       5b47de04e4b03c859495a0f7
9        5cb4d5d0e4b0bd6bed7869fa
7        5ad08c66e4b01bf0eefaf887
4        5aa291c2e4b0c65fcc798ce8
1        5acdf7d8e4b014cba3b1724b
Name: customer_id, Length: 2238, dtype: object

In [116]:
#busque los res.partner que es en donde se van a almacenar los customer de Chargebee

i2 = odoo.search_read('res.partner', [], [], limit=None)  # Establecer limit=None para obtener todos los registros
partner = pd.DataFrame(i2)

partner

,id,avatax_unique_code,website_id,website_published,is_published,can_publish,website_url,message_is_follower,message_follower_ids,message_partner_ids,...,l10n_es_edi_facturae_ac_role_type_ids,l10n_es_edi_facturae_ac_physical_gln,l10n_es_edi_facturae_ac_logical_operational_point,sale_type,subscription_count,last_website_so_id,l10n_cl_delivery_guide_price,arba_alicuot_ids,drei,default_regimen_ganancias_id
0,86,Contact 86,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
1,87,Contact 87,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
2,67,Contact 67,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
3,46105,Contact 46105,False,False,False,True,#,True,[135421],[121],...,[],False,False,False,0,False,sale_order,[],False,False
4,46838,Contact 46838,False,False,False,True,#,True,[136154],[121],...,[],False,False,False,0,False,sale_order,[],False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2364,69,Contact 69,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
2365,77,Contact 77,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
2366,63,Contact 63,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
2367,76,Contact 76,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False


In [117]:
#deshacerme de los nombres duplicados del modelo res.partner
all_partners=partner.drop_duplicates('name')
all_partners

,id,avatax_unique_code,website_id,website_published,is_published,can_publish,website_url,message_is_follower,message_follower_ids,message_partner_ids,...,l10n_es_edi_facturae_ac_role_type_ids,l10n_es_edi_facturae_ac_physical_gln,l10n_es_edi_facturae_ac_logical_operational_point,sale_type,subscription_count,last_website_so_id,l10n_cl_delivery_guide_price,arba_alicuot_ids,drei,default_regimen_ganancias_id
0,86,Contact 86,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
1,87,Contact 87,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
2,67,Contact 67,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
3,46105,Contact 46105,False,False,False,True,#,True,[135421],[121],...,[],False,False,False,0,False,sale_order,[],False,False
4,46838,Contact 46838,False,False,False,True,#,True,[136154],[121],...,[],False,False,False,0,False,sale_order,[],False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2364,69,Contact 69,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
2365,77,Contact 77,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
2366,63,Contact 63,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
2367,76,Contact 76,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False


In [118]:
#visualizo el id y elcampo en donde se muestra el nombre del customer
i3 = partner[['name', 'id']]

In [119]:
i3

,name,id
0,(AR) Exento,86
1,(AR) Monotributista,87
2,0h3n-rijaune@yahoo.example.com,67
3,16CXyqUDJkqSi1S5N,46105
4,535680f7e4b03a7c45f66703,46838
...,...,...
2364,the.jose@gmail.example.com,69
2365,the.real.shane@gmail.example.com,77
2366,thing.thang.thong@gmail.example.com,63
2367,tina.turner@gmail.example.com,76


In [31]:
#luego ejecute una insercion en ese modelo y funciono exitosamente (el campo que vi que podia llenar con el string de un customer_id)
#era ese campo name de todos los que tiene el dataframe


#odoo.create('res.partner', [{
#    'name': '5dde8435e4b098a27bbd4def'
#}])

In [120]:
#veo los tipos de datos del dataframe de los partners solo con nombre y id
print_column_types(i3)

name 	 object
id 	 int64


In [33]:
#genere un dataframe sobre los campos con tipo de dato object que es un equivalente a un string y poder encontrar que campo habia que
# llenar para que me se pudiera ver el nombre del partner en la interfaz grafica


#many = partner.select_dtypes(include=['object'])
#many

In [121]:
#empiezo a generar el codigo para hacer una insercion de los customers de Chargebee en Odoo
# Inicializar una lista para almacenar el historial de inserciones
insertion_history = []

# Función para insertar nuevos registros en res.partner
def insert_new_partners(customers, execution_id):
    # Leer todos los registros actuales del modelo res.partner
    existing_partners = odoo.search_read('res.partner', [], ['name'])
    
    # Extraer los nombres de los partners existentes
    existing_partner_names = {partner['name'] for partner in existing_partners}
    
    # Obtener la lista de nombres de customers del DataFrame
    customer_ids = customers.tolist()
    
    # Crear una lista de diccionarios para los nuevos registros a insertar
    # Solo insertar aquellos que no existen en el modelo res.partner
    records_to_insert = [{'name': customer_id} for customer_id in customer_ids if customer_id not in existing_partner_names]
    
    # Realizar la inserción masiva uno por uno para evitar sobrecarga en la llamada a Odoo.create
    for record in records_to_insert:
        odoo.create('res.partner', [record])
    
    # Guardar los nuevos registros en el historial con un ID de ejecución
    insertion_history.append({
        'execution_id': execution_id,
        'new_records': records_to_insert
    })
    
    # Mostrar cuántos registros nuevos fueron insertados
    print(f"Se insertaron {len(records_to_insert)} nuevos registros en res.partner.")


In [122]:

# Función para obtener los registros insertados en una ejecución específica
def get_inserted_records(execution_id):
    # Buscar la ejecución en el historial
    for execution in insertion_history:
        if execution['execution_id'] == execution_id:
            # Convertir la lista de diccionarios en un DataFrame
            df = pd.DataFrame(execution['new_records'])
            return df
    
    # Si no se encuentra la ejecución
    print(f"No se encontraron registros para la ejecución con ID: {execution_id}")
    return None

In [209]:
# Insertar nuevos registros con un ID de ejecución '1'
insert_new_partners(customers, execution_id='1')

Se insertaron 2251 nuevos registros en res.partner.


In [68]:
# Obtener los registros insertados en la ejecución '1'
#esto lo hago para tener el registro de que data acabo de importar para luego tener ese bloque de data para generar las invoices
# o incluso borrarlos cuando lleguemos a otras metodologias

df_execution_1 = get_inserted_records('4')
d_exec = pd.DataFrame(df_execution_1)
d_exec

No se encontraron registros para la ejecución con ID: 4


""


In [125]:
#genere este codigo para las ocasiones en las que hay que realizar la eliminacion de muchos registros sin que afecte a los que hay que 
# proteger ya que al hacer el llamado directo y luego borrar es lo que podria sabotear el trabajo debido a un error humano jaja
# Obtener todos los registros del modelo res.partner, ordenados por id de forma descendente y con los campos 'id' y 'name'
i5 = odoo.search_read('res.partner', [], ['id', 'name'], limit=None)

# Convertir los resultados a un DataFrame
partner2 = pd.DataFrame(i5)

# Ordenar el DataFrame por la columna 'id' en orden descendente y seleccionar la cantidad de los ultimos registros que hay que ver
last = partner2.sort_values(by='id', ascending=False).head(2251)

# Mostrar el DataFrame con los últimos registros mostrando 'id' y 'name'
last_rows_df = pd.DataFrame(last)
last_rows_df

,id,name
179,46869,5a0c58c9e4b031a2c96d319c
5,46868,536109b2e4b0ae650c483374
6,46867,53c42a63e4b08c5d83b878ff
27,46866,5727680de4b06a6e4222f0c8
215,46865,5a71ba83e4b07b95e31fc272
...,...,...
344,44623,5b47de04e4b03c859495a0f7
559,44622,5cb4d5d0e4b0bd6bed7869fa
265,44621,5ad08c66e4b01bf0eefaf887
240,44620,5aa291c2e4b0c65fcc798ce8


In [126]:
last_rows_df.loc[last_rows_df['name'] == '664561c79c70d34ec0e801ec']

,id,name
2117,45079,664561c79c70d34ec0e801ec


In [127]:
#aqui llame al dataframe de los registros de res.partner para ver solamente el nombre y id
p_id = partner[['id', 'name']]
p_id_df = pd.DataFrame(p_id)
p_id_df

,id,name
0,86,(AR) Exento
1,87,(AR) Monotributista
2,67,0h3n-rijaune@yahoo.example.com
3,46105,16CXyqUDJkqSi1S5N
4,46838,535680f7e4b03a7c45f66703
...,...,...
2364,69,the.jose@gmail.example.com
2365,77,the.real.shane@gmail.example.com
2366,63,thing.thang.thong@gmail.example.com
2367,76,tina.turner@gmail.example.com


In [229]:
#despues uni el dataframe que me mostraba el resultado de los customers que agregue 
# para que hiciera una union con el dataframe de los partners ya que al ser exitosa la insercion de los customers en odoo
# ahora esos customers de chargeBee ya estan en Odoo



#new_partners_df = pd.merge(p_id_df, d_exec, on='name', how='inner')
#new_partners_df

In [230]:
#ahora voy a eliminar esos registros ya que aun no habia corregido los customers ya que hay duplicados


#partner_id_to_delete = 131  # Reemplazar con el ID real que hay que eliminar

# Realizar la operación de eliminación


#odoo.execute('res.partner', 'unlink', [partner_id_to_delete])


In [204]:
# Obtener los IDs de los últimos  registros
partner_ids_to_delete = last['id'].tolist()

# Eliminar los registros utilizando el método 'unlink' de Odoo
if partner_ids_to_delete:
    odoo.execute('res.partner', 'unlink', [partner_ids_to_delete])

# Mostrar el número de registros eliminados
print(f"Se han eliminado {len(partner_ids_to_delete)} registros de res.partner.")

Se han eliminado 2254 registros de res.partner.


In [271]:
#Buscar un registro de los recien ingresados partners por medio de su id

#last_rows_df.loc[last_rows_df['id'] == 25762]

In [270]:
#Este codigo estaba de prueba para verficar que no se dupliquen los registros y el resultado fue exitoso

#dd = last_rows_df.drop_duplicates('name')
#dd

In [128]:
#unir los dataframes que muestran los ultimos partneres con uno de llamado a los partners para confirmar que los partners coincidad 
# 
 
last_partners_df = pd.merge(p_id_df, last_rows_df, on='name', how='inner')
last_partners_df.drop(columns=['id_y'], inplace=True)
last_partners_df.sort_values(by='id_x', ascending=False)
last_partners_df 

,id_x,name
0,46105,16CXyqUDJkqSi1S5N
1,46838,535680f7e4b03a7c45f66703
2,46868,536109b2e4b0ae650c483374
3,46867,53c42a63e4b08c5d83b878ff
4,46864,540f3b9ee4b0492f27113fe7
...,...,...
2246,46733,66ccdda3b1c5d119cf1a2412
2247,46709,66cf6e434427b90002f38734
2248,46706,66cf6f0374ecf760b7a5b258
2249,46768,66d0a6efa9c6e736847a56e6


In [129]:
#renombrar el campo name de res.partner para que se vea como el id del partner o de customer
r_last_rows_df = last_rows_df.rename(columns={'name': 'customer_id'})
r_last_rows_df.sort_values(by='id', ascending=False)
r_last_rows_df

,id,customer_id
179,46869,5a0c58c9e4b031a2c96d319c
5,46868,536109b2e4b0ae650c483374
6,46867,53c42a63e4b08c5d83b878ff
27,46866,5727680de4b06a6e4222f0c8
215,46865,5a71ba83e4b07b95e31fc272
...,...,...
344,44623,5b47de04e4b03c859495a0f7
559,44622,5cb4d5d0e4b0bd6bed7869fa
265,44621,5ad08c66e4b01bf0eefaf887
240,44620,5aa291c2e4b0c65fcc798ce8


In [131]:
#unir el dataframe anterior con el de order que son todas las invoices de Chargebee y que estos coincidad para luego insertar los
#datos de Chargebee necesarios para migrar a Odoo, terminando en un dataframe que tiene el partner_id y los campos de invoices de
#Chargebee, esto lo hago para que en un solo dataframe tenga toda la data necesaria para posteriormente introducirla en account.move


partners_Odoo_invoices = pd.merge(r_last_rows_df, order, on='customer_id', how='right')
partners_Odoo_invoices.rename(columns={'id_x': 'partner_id'}, inplace=True)
partners_Odoo_invoices.rename(columns={'id_y': 'invoice_id'}, inplace=True)
partners_Odoo_invoices.sort_values(by='partner_id', ascending=False)
partners_Odoo_invoices

,partner_id,customer_id,values,sub_types,dependant_types,invoice_id,po_number,subscription_id,recurring,status,...,line_item_tiers_quantity_used,line_item_tiers_unit_amount,line_item_tiers_object,line_item_tiers_line_item_id,issued_credit_notes_cn_id,issued_credit_notes_cn_reason_code,issued_credit_notes_cn_create_reason_code,issued_credit_notes_cn_date,issued_credit_notes_cn_total,issued_credit_notes_cn_status
0,46869.0,5a0c58c9e4b031a2c96d319c,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,1mkVvlDQwVxVgL8H4,True,paid,...,3.0,700.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
1,46869.0,5a0c58c9e4b031a2c96d319c,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,1mkVvlDQwVxVgL8H4,True,paid,...,1.0,0.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
2,46869.0,5a0c58c9e4b031a2c96d319c,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,1mkVvlDQwVxVgL8H4,True,paid,...,3.0,700.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
3,46869.0,5a0c58c9e4b031a2c96d319c,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,1mkVvlDQwVxVgL8H4,True,paid,...,1.0,0.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
4,46868.0,536109b2e4b0ae650c483374,"{'id': '187913', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187913,Argentina - Transferencia Bancaria (Leadaki),16CY4nUBd1nGm7OLS,True,paid,...,5.0,0.0,line_item_tier,li_Azyk2WUN5VwXQ7GuX,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10474,44620.0,5aa291c2e4b0c65fcc798ce8,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,6or0jU5PxxMNEhuk,True,paid,...,1.0,0.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
10475,44620.0,5aa291c2e4b0c65fcc798ce8,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,6or0jU5PxxMNEhuk,True,paid,...,3.0,1000.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
10476,44620.0,5aa291c2e4b0c65fcc798ce8,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,6or0jU5PxxMNEhuk,True,paid,...,3.0,1000.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
10477,44619.0,5acdf7d8e4b014cba3b1724b,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,169uzAU4TrwKK33PY,True,paid,...,1.0,0.0,line_item_tier,li_6olL2UKBV7cm22we,NaN,NaN,NaN,NaN,NaN,NaN


In [233]:
#order


,id,po_number,customer_id,subscription_id,recurring,status,price_type,date,due_date,net_term_days,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
0,185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,tax_exclusive,2024-08-01 05:08:00-03:00,2024-08-01 05:08:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,tax_exclusive,2024-08-01 07:17:00-03:00,2024-08-01 07:17:00-03:00,0,...,NaN,line_item_discount,li_Azz6aoUKC1bN81M2q,item_level_coupon,2000.0,Migra2024Premium2Users,Migra2024Premium2Users,NaN,NaN,NaN
2,185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,True,paid,tax_exclusive,2024-08-01 07:39:00-03:00,2024-08-01 07:39:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,185395,Argentina - Transferencia Bancaria,5cb4d5d0e4b0bd6bed7869fa,16CY2RUEdBvEE1YCI,True,paid,tax_exclusive,2024-08-01 07:54:00-03:00,2024-08-01 07:54:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,185396,Chile,5b47de04e4b03c859495a0f7,Izy9TIBR04obyPV06,True,paid,tax_exclusive,2024-08-01 08:05:08-03:00,2024-08-01 08:05:08-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,187910,Argentina - Transferencia Bancaria (Leadaki),5a71ba83e4b07b95e31fc272,AzqBxKUBeW38u8EGG,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2453,187911,Argentina - Transferencia Bancaria (Leadaki),5727680de4b06a6e4222f0c8,AzyhzvUBeSCea8nZZ,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2454,187912,Argentina - Transferencia Bancaria (Leadaki),53c42a63e4b08c5d83b878ff,AzqBxKUBeN2WG88Ao,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2446,187913,Argentina - Transferencia Bancaria (Leadaki),536109b2e4b0ae650c483374,16CY4nUBd1nGm7OLS,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
#aqui lo escribo el codigo de insercion para que se vea que funciona al hacerla registro por registro

#aqui el codigo que genera una invoice de clientes 


#odoo.create('account.move', [{
 #   'move_type': 'out_invoice',  # Factura de cliente
  #  'partner_id': 138,  # ID del cliente
   # 'company_id': 1,  # ID de la compañía
    #'website_id': 1,  # ID del sitio web
    #'campaign_id': False,
    #'source_id': False,
    #'medium_id': False,
    #'activity_ids': [],
    #'amount_total': 28.75,
    #'amount_residual': 28.75,
    #'amount_total_signed': 28.75,
    #'amount_total_in_currency_signed': 28.75,
    #'amount_residual_signed': 28.75,
    #'activity_state': True,
#}])

166

In [55]:
#aqui tengo otro codigo de prueba
#
#odoo.create('account.move', [{
 #           'move_type': 'out_invoice',
 #          'partner_id': 34827,
 #           'journal_id': 1,
 #           'currency_id' : 1,
 #           'amount_untaxed': 0.00,
 #           'amount_untaxed_signed': 0.00,
 #           'amount_tax': 0.00,
 #           'amount_tax_signed': 0.00
 #   }])

66064

In [132]:
#aqui yo busque todos los registros de tipo de moneda que tenga registrado account.move para tambien tenerlo en cuenta al insertar las
#invoices de chargebee que esten en pesos o dolares y luego agregarlos a odoo

cur = odoo.search_read('account.move', [], ['currency_id'], limit=None)

# Convertir los resultados a un DataFrame
currency_c = pd.DataFrame(cur)
currency_c

,id,currency_id
0,164,"[1, USD]"
1,161,"[1, USD]"
2,163,"[1, USD]"
3,160,"[1, USD]"
4,159,"[1, USD]"
...,...,...
2493,19,"[1, USD]"
2494,18,"[1, USD]"
2495,17,"[1, USD]"
2496,11,"[1, USD]"


In [276]:
# Revisaso la estructura de los datos del campo para poderlo separar en 2, luego unirlo con el campo de currency de chargebee y poder 
# unir ambos dataframes por medio del tipo de moneda mas adelante

print(currency_c['currency_id'].head(10))  # Revisar si los valores están en el formato correcto

0    [1, USD]
1    [1, USD]
2    [1, USD]
3    [1, USD]
4    [1, USD]
5    [1, USD]
6    [1, USD]
7    [1, USD]
8    [1, USD]
9    [1, USD]
Name: currency_id, dtype: object


In [40]:
#aqui busco entre todas las invoices de chargebee, del dataframe en el que ordene todos por el id de la invoice de chargebee
#  para ver que invoices de chargebee tienen otra moneda diferente a dolares 
order.loc[order['currency_code'] != 'USD']

,id,po_number,customer_id,subscription_id,recurring,status,price_type,date,due_date,net_term_days,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
1810,187266,cuenta test,61c391b3fa6287002a7c1071,AzyXUBSsMO62lkOo,True,paid,tax_exclusive,2024-08-22 18:29:51-03:00,2024-08-22 18:29:51-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2300,187761,test,62bcbd7c9b9f80002a6b11b8,AzqSIKTABORoW3SxD,True,paid,tax_exclusive,2024-08-29 18:02:06-03:00,2024-08-29 18:02:06-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [133]:
#Este codigo lo use para separar el campo currency_id de account.move en 2 ya que aparece su id como un array que tiene un int y un 
# string como se mostro en los codigos anteriores que llaman a ese campo

# Asegurarse de que currency_id sean listas y cada lista tenga dos elementos
currency_c['currency_id'] = currency_c['currency_id'].apply(lambda x: x if isinstance(x, list) and len(x) == 2 else [None, None])

# Separar la columna currency_id en dos columnas: currency_num y currency_name
currency_c[['currency_num', 'currency_code']] = pd.DataFrame(currency_c['currency_id'].tolist(), index=currency_c.index)

# Convertir el campo currency_num a tipo numérico
currency_c['currency_num'] = pd.to_numeric(currency_c['currency_num'], errors='coerce')

# Verificar el resultado refinado con las nuevas columnas
currency_refined = currency_c[['currency_num', 'currency_code']]
currency_refined

,currency_num,currency_code
0,1,USD
1,1,USD
2,1,USD
3,1,USD
4,1,USD
...,...,...
2493,1,USD
2494,1,USD
2495,1,USD
2496,1,USD


In [134]:
#una vez se determinan los campos de moneda, se eliminan los duplicados para ver que tipo de moneda tiene registrado odoo en las facturas
currency_re_defined = currency_refined.drop_duplicates('currency_code')
currency_re_defined

,currency_num,currency_code
0,1,USD


In [135]:
partners_to_odoo_invoices = pd.merge(currency_re_defined, partners_Odoo_invoices, on='currency_code', how='inner')
partners_to_odoo_invoices

,currency_num,currency_code,partner_id,customer_id,values,sub_types,dependant_types,invoice_id,po_number,subscription_id,...,line_item_tiers_quantity_used,line_item_tiers_unit_amount,line_item_tiers_object,line_item_tiers_line_item_id,issued_credit_notes_cn_id,issued_credit_notes_cn_reason_code,issued_credit_notes_cn_create_reason_code,issued_credit_notes_cn_date,issued_credit_notes_cn_total,issued_credit_notes_cn_status
0,1,USD,46869.0,5a0c58c9e4b031a2c96d319c,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,1mkVvlDQwVxVgL8H4,...,3.0,700.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
1,1,USD,46869.0,5a0c58c9e4b031a2c96d319c,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,1mkVvlDQwVxVgL8H4,...,1.0,0.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
2,1,USD,46869.0,5a0c58c9e4b031a2c96d319c,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,1mkVvlDQwVxVgL8H4,...,3.0,700.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
3,1,USD,46869.0,5a0c58c9e4b031a2c96d319c,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,1mkVvlDQwVxVgL8H4,...,1.0,0.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
4,1,USD,46868.0,536109b2e4b0ae650c483374,"{'id': '187913', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187913,Argentina - Transferencia Bancaria (Leadaki),16CY4nUBd1nGm7OLS,...,5.0,0.0,line_item_tier,li_Azyk2WUN5VwXQ7GuX,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10472,1,USD,44620.0,5aa291c2e4b0c65fcc798ce8,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,6or0jU5PxxMNEhuk,...,1.0,0.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
10473,1,USD,44620.0,5aa291c2e4b0c65fcc798ce8,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,6or0jU5PxxMNEhuk,...,3.0,1000.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
10474,1,USD,44620.0,5aa291c2e4b0c65fcc798ce8,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,6or0jU5PxxMNEhuk,...,3.0,1000.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
10475,1,USD,44619.0,5acdf7d8e4b014cba3b1724b,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,169uzAU4TrwKK33PY,...,1.0,0.0,line_item_tier,li_6olL2UKBV7cm22we,NaN,NaN,NaN,NaN,NaN,NaN


In [136]:
# Convertir el campo 'partner_id' a enteros, asegurando que no queden decimales ya que al unirlos por alguna razon los mostraba con .0 al final de cada id
partners_to_odoo_invoices['partner_id'] = partners_to_odoo_invoices['partner_id'].fillna(0).astype(int)

# Mostrar el DataFrame para verificar los cambios
partners_to_odoo_invoices


,currency_num,currency_code,partner_id,customer_id,values,sub_types,dependant_types,invoice_id,po_number,subscription_id,...,line_item_tiers_quantity_used,line_item_tiers_unit_amount,line_item_tiers_object,line_item_tiers_line_item_id,issued_credit_notes_cn_id,issued_credit_notes_cn_reason_code,issued_credit_notes_cn_create_reason_code,issued_credit_notes_cn_date,issued_credit_notes_cn_total,issued_credit_notes_cn_status
0,1,USD,46869,5a0c58c9e4b031a2c96d319c,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,1mkVvlDQwVxVgL8H4,...,3.0,700.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
1,1,USD,46869,5a0c58c9e4b031a2c96d319c,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,1mkVvlDQwVxVgL8H4,...,1.0,0.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
2,1,USD,46869,5a0c58c9e4b031a2c96d319c,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,1mkVvlDQwVxVgL8H4,...,3.0,700.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
3,1,USD,46869,5a0c58c9e4b031a2c96d319c,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,1mkVvlDQwVxVgL8H4,...,1.0,0.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
4,1,USD,46868,536109b2e4b0ae650c483374,"{'id': '187913', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187913,Argentina - Transferencia Bancaria (Leadaki),16CY4nUBd1nGm7OLS,...,5.0,0.0,line_item_tier,li_Azyk2WUN5VwXQ7GuX,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10472,1,USD,44620,5aa291c2e4b0c65fcc798ce8,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,6or0jU5PxxMNEhuk,...,1.0,0.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
10473,1,USD,44620,5aa291c2e4b0c65fcc798ce8,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,6or0jU5PxxMNEhuk,...,3.0,1000.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
10474,1,USD,44620,5aa291c2e4b0c65fcc798ce8,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,6or0jU5PxxMNEhuk,...,3.0,1000.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
10475,1,USD,44619,5acdf7d8e4b014cba3b1724b,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,169uzAU4TrwKK33PY,...,1.0,0.0,line_item_tier,li_6olL2UKBV7cm22we,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
renamed_partners_to_odoo = partners_to_odoo_invoices.rename(columns={'id_x': 'partner_id'})
renamed_partners_to_odoo

,currency_num,currency_code,partner_id,customer_id,values,sub_types,dependant_types,invoice_id,po_number,subscription_id,...,line_item_tiers_quantity_used,line_item_tiers_unit_amount,line_item_tiers_object,line_item_tiers_line_item_id,issued_credit_notes_cn_id,issued_credit_notes_cn_reason_code,issued_credit_notes_cn_create_reason_code,issued_credit_notes_cn_date,issued_credit_notes_cn_total,issued_credit_notes_cn_status
0,1,USD,46869,5a0c58c9e4b031a2c96d319c,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,1mkVvlDQwVxVgL8H4,...,3.0,700.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
1,1,USD,46869,5a0c58c9e4b031a2c96d319c,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,1mkVvlDQwVxVgL8H4,...,1.0,0.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
2,1,USD,46869,5a0c58c9e4b031a2c96d319c,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,1mkVvlDQwVxVgL8H4,...,3.0,700.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
3,1,USD,46869,5a0c58c9e4b031a2c96d319c,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,1mkVvlDQwVxVgL8H4,...,1.0,0.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
4,1,USD,46868,536109b2e4b0ae650c483374,"{'id': '187913', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187913,Argentina - Transferencia Bancaria (Leadaki),16CY4nUBd1nGm7OLS,...,5.0,0.0,line_item_tier,li_Azyk2WUN5VwXQ7GuX,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10472,1,USD,44620,5aa291c2e4b0c65fcc798ce8,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,6or0jU5PxxMNEhuk,...,1.0,0.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
10473,1,USD,44620,5aa291c2e4b0c65fcc798ce8,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,6or0jU5PxxMNEhuk,...,3.0,1000.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
10474,1,USD,44620,5aa291c2e4b0c65fcc798ce8,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,6or0jU5PxxMNEhuk,...,3.0,1000.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
10475,1,USD,44619,5acdf7d8e4b014cba3b1724b,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,169uzAU4TrwKK33PY,...,1.0,0.0,line_item_tier,li_6olL2UKBV7cm22we,NaN,NaN,NaN,NaN,NaN,NaN


In [402]:
#renamed_partners_to_odoo.loc[renamed_partners_to_odoo['partner_id'] == 0]

In [224]:
#visualizar si algun registro de facturas de agosto tiene impuestos

#partners_to_odoo_invoices.loc[partners_to_odoo_invoices['tax_amount'] > 0]

In [404]:
#aqui llame a un dataframe que declare mas abajo, ignora esto

# date_df

In [138]:
partners_to_odoo_invoices['date_from']

0        2024-09-01
1        2024-09-01
2        2024-09-01
3        2024-09-01
4        2024-08-30
            ...    
10472    2024-08-01
10473    2024-08-01
10474    2024-08-01
10475    2024-08-01
10476    2024-08-01
Name: date_from, Length: 10477, dtype: object

In [139]:
# Filtrar las columnas que contienen 'date' en su nombre
date_columns = [col for col in order.columns if 'date' in col.lower()]

# Crear un nuevo dataframe solo con las columnas que contienen 'date'
date_df = order[date_columns]

# Mostrar el nuevo dataframe filtrado
date_df


,date,due_date,updated_at,expected_payment_date,date_from,date_to,linked_payments_txn_date,issued_credit_notes_cn_date
10478,2024-09-01 00:06:31-03:00,2024-09-01 00:06:31-03:00,2024-09-01 00:06:35-03:00,NaN,2024-09-01,2024-10-01 00:06:31-03:00,1.725160e+09,NaN
10477,2024-09-01 00:06:31-03:00,2024-09-01 00:06:31-03:00,2024-09-01 00:06:35-03:00,NaN,2024-09-01,2024-10-01 00:06:31-03:00,1.725160e+09,NaN
10476,2024-09-01 00:06:31-03:00,2024-09-01 00:06:31-03:00,2024-09-01 00:06:35-03:00,NaN,2024-09-01,2024-10-01 00:06:31-03:00,1.725160e+09,NaN
10475,2024-09-01 00:06:31-03:00,2024-09-01 00:06:31-03:00,2024-09-01 00:06:35-03:00,NaN,2024-09-01,2024-10-01 00:06:31-03:00,1.725160e+09,NaN
9723,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,2024-09-01 00:00:35-03:00,NaN,2024-08-30,2024-08-30 12:11:17-03:00,NaN,NaN
...,...,...,...,...,...,...,...,...
2,2024-08-01 07:17:00-03:00,2024-08-01 07:17:00-03:00,2024-08-13 13:55:55-03:00,NaN,2024-08-01,2024-09-01 07:17:00-03:00,1.723050e+09,NaN
5,2024-08-01 07:17:00-03:00,2024-08-01 07:17:00-03:00,2024-08-13 13:55:55-03:00,NaN,2024-08-01,2024-09-01 07:17:00-03:00,1.723050e+09,NaN
3,2024-08-01 07:17:00-03:00,2024-08-01 07:17:00-03:00,2024-08-13 13:55:55-03:00,NaN,2024-08-01,2024-09-01 07:17:00-03:00,1.723050e+09,NaN
1,2024-08-01 05:08:00-03:00,2024-08-01 05:08:00-03:00,2024-08-23 14:55:19-03:00,NaN,2024-08-01,2024-09-01 05:08:00-03:00,1.724349e+09,NaN


In [141]:
# Filtrar las columnas que contienen 'date' en su nombre
tax_columns = [col for col in inv.columns if 'tax' in col.lower()]

# Crear un nuevo dataframe solo con las columnas que contienen 'date'
tax = inv[tax_columns]

# Mostrar el nuevo dataframe filtrado
tax


,is_tax_computed_externally,is_avatax,tax_cash_basis_rec_id,tax_cash_basis_origin_move_id,tax_cash_basis_created_move_ids,always_tax_exigible,tax_calculation_rounding_method,amount_untaxed,amount_tax,amount_untaxed_signed,...,tax_totals,tax_lock_date_message,tax_country_id,tax_country_code,avatax_unique_code,avatax_tax_date,tax_closing_end_date,tax_report_control_error,tax_closing_alert,tax_closing_show_multi_closing_warning
0,False,False,False,False,[],False,round_per_line,0.0,0.00,0.0,...,"{'amount_untaxed': 0.0, 'amount_total': 0.0, '...",False,"[233, United States]",US,Journal Entry 164,False,False,False,False,False
1,False,False,False,False,[],False,round_per_line,25.0,3.75,25.0,...,"{'amount_untaxed': 25.0, 'amount_total': 28.75...",False,"[233, United States]",US,Journal Entry 161,False,False,False,False,False
2,False,False,False,False,[],False,round_per_line,0.0,0.00,0.0,...,"{'amount_untaxed': 0.0, 'amount_total': 0.0, '...",False,"[233, United States]",US,Journal Entry 163,False,False,False,False,False
3,False,False,False,False,[],False,round_per_line,0.0,0.00,0.0,...,"{'amount_untaxed': 0.0, 'amount_total': 0.0, '...",False,"[233, United States]",US,Journal Entry 160,False,False,False,False,False
4,False,False,False,False,[],False,round_per_line,0.0,0.00,0.0,...,"{'amount_untaxed': 0.0, 'amount_total': 0.0, '...",False,"[233, United States]",US,Journal Entry 159,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2493,False,False,False,False,[],True,round_per_line,0.0,0.00,0.0,...,False,False,"[233, United States]",US,Journal Entry 19,False,False,False,False,False
2494,False,False,False,False,[],True,round_per_line,0.0,0.00,0.0,...,False,False,"[233, United States]",US,Journal Entry 18,False,False,False,False,False
2495,False,False,False,False,[],True,round_per_line,0.0,0.00,0.0,...,False,False,"[233, United States]",US,Journal Entry 17,False,False,False,False,False
2496,False,False,False,False,[],False,round_per_line,541.1,81.17,541.1,...,"{'amount_untaxed': 541.1, 'amount_total': 622....",False,"[233, United States]",US,Journal Entry 11,False,False,False,False,False


In [228]:
#tax.loc[tax['amount_total'] == 0]

In [143]:
#una vez teniendo el modelo de res.partner e identificado los datos que debo insertar, generare la insercion de invoices en Odoo
#solamente como prueba ya que unicamente voy a incluir unos campos a las facturas pero lo mas importante en este caso es ingresar el
# partner_id de cada partner que ya enliste despues de ingresar el monton de registros de Chargebee en Odoo y asi de una vez saber a que
#partners debo ingresar en las invoices 
# Función para insertar facturas en account.move con validación de datos
import signal
import time

# Variable global para indicar si hubo una interrupción manual o del sistema
interrupted = False

# Función para manejar señales de interrupción (como CTRL+C o kill)
def handle_interrupt(signum, frame):
    global interrupted
    print(f"Señal {signum} recibida. Iniciando rollback...")
    interrupted = True  # Marcar interrupción
    raise Exception("Proceso interrumpido manualmente o por el sistema.")

# Conectar las señales de interrupción al manejador de interrupciones
signal.signal(signal.SIGINT, handle_interrupt)   # CTRL+C
signal.signal(signal.SIGTERM, handle_interrupt)  # kill command

def insert_invoices_with_rollback(renamed_partners_to_odoo, execution_id):
    global interrupted
    records_to_insert = []
    inserted_record_ids = []

    # Iterar sobre el DataFrame
    for _, row in renamed_partners_to_odoo.iterrows():
        record = {
            'move_type': 'out_invoice',
            'partner_id': row['partner_id'],
            'journal_id': 1,
            'currency_id': row['currency_num'],
            'date': row['date_from'].strftime('%Y-%m-%d'),
            'invoice_date': row['date_from'].strftime('%Y-%m-%d'),
            'amount_tax': row['tax_amount']
        }

        if 'partner_id' not in record or not record['partner_id']:
            print(f"Registro inválido: {record}")
            continue

        records_to_insert.append(record)

    if records_to_insert:
        try:
            # Realizar inserciones en un bucle
            for record in records_to_insert:
                if interrupted:
                    raise Exception("Interrupción detectada antes de insertar.")
                
                # Crear un registro (necesitamos envolver el diccionario en una lista)
                result = odoo.execute('account.move', 'create', [record])
                inserted_record_ids.append(result)

            insertion_history.append({
                'execution_id': execution_id,
                'new_records': inserted_record_ids
            })

            print(f"Se insertaron {len(inserted_record_ids)} nuevas facturas en account.move.")

        except Exception as e:
            print(f"Error durante la inserción: {str(e)}")

            # Intentar eliminar los registros ya insertados en caso de error
            if inserted_record_ids:
                try:
                    print(f"Intentando eliminar los registros insertados debido al error...")
                    odoo.execute('account.move', 'unlink', inserted_record_ids)
                    print("Registros eliminados correctamente.")
                except Exception as rollback_error:
                    print(f"Error al intentar eliminar registros: {str(rollback_error)}")

    else:
        print("No se encontraron registros válidos para insertar en account.move.")


In [356]:
# Prueba de la función de inserción
insert_invoices_with_rollback(renamed_partners_to_odoo, execution_id='1')

Se insertaron 2459 nuevas facturas en account.move.


In [146]:
#codigo para mostrar los ultimos registros en el modelo, tomando en cuenta la cantidad d eregistros que se insertaron en el Bulk insert anterior

#amount_untaxed amount_tax
i6 = odoo.search_read('account.move', [], ['id', 'partner_id', 'currency_id', 'amount_untaxed', 'amount_tax'], limit=None)

# Convertir los resultados a un DataFrame
inv_df = pd.DataFrame(i6)

# Ordenar el DataFrame por la columna 'id' en orden descendente y seleccionar la cantidad de los ultimos registros que hay que ver
last_inv = inv_df.sort_values(by='id', ascending=False).head(2459)

# Mostrar el DataFrame con los últimos registros mostrando 'id' y 'name'
last_invoices = pd.DataFrame(last_inv)
last_invoices
#habian 39 antes de la insercion

,id,partner_id,currency_id,amount_untaxed,amount_tax
21,97870,"[46869, 5a0c58c9e4b031a2c96d319c]","[1, USD]",129.0,0.0
22,97869,"[46868, 536109b2e4b0ae650c483374]","[1, USD]",42.0,0.0
23,97868,"[46867, 53c42a63e4b08c5d83b878ff]","[1, USD]",0.0,0.0
24,97867,"[46866, 5727680de4b06a6e4222f0c8]","[1, USD]",1200.0,0.0
25,97866,"[46865, 5a71ba83e4b07b95e31fc272]","[1, USD]",30.0,0.0
...,...,...,...,...,...
2407,95416,"[44623, 5b47de04e4b03c859495a0f7]","[1, USD]",3900.0,0.0
2408,95415,"[44622, 5cb4d5d0e4b0bd6bed7869fa]","[1, USD]",120.0,0.0
2409,95414,"[44621, 5ad08c66e4b01bf0eefaf887]","[1, USD]",89.0,0.0
2410,95413,"[44620, 5aa291c2e4b0c65fcc798ce8]","[1, USD]",0.0,0.0


In [324]:
#last_invoices.loc[last_invoices['amount_tax'] > 0]

In [290]:
#pppppp = order[['customer_id', 'total']]
#pppppp

In [291]:
#pppppp.loc[pppppp['customer_id'] == '60c3d852b0d246002afdde1b']

In [292]:
#confirmar que no tengan impuestos ya que las facturas de agosto no tienen impuestos

#last_invoices.loc[last_invoices['amount_untaxed'] > 0]

In [352]:
# Obtener los IDs de los últimos  registros de account.move y eliminarlos en base al lsitado de los ultimos ids o de un bloque de
#registros a eliminar
ids_to_delete = last_inv['id'].tolist()

# Eliminar los registros utilizando el método 'unlink' de Odoo
if ids_to_delete:
    odoo.execute('account.move', 'unlink', [ids_to_delete])

# Mostrar el número de registros eliminados
print(f"Se han eliminado {len(ids_to_delete)} registros de account.move.")

Se han eliminado 210 registros de account.move.


In [194]:
#mostrar todos los campos de account.move

ii = odoo.search_read('account.move', [], [], limit=None)
ii_df = pd.DataFrame(ii)

In [195]:
ii_df

,id,campaign_id,source_id,medium_id,is_tax_computed_externally,is_avatax,sequence_prefix,sequence_number,activity_ids,activity_state,...,l10n_ar_afip_asoc_period_start,l10n_ar_afip_asoc_period_end,l10n_ar_boarding_permission_ids,timesheet_ids,timesheet_count,timesheet_encode_uom_id,timesheet_total_duration,document_type_internal_type,invoice_validation_type,l10n_ar_edi_warning
0,164,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
1,161,False,False,False,False,False,INV/2024/,11,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2,163,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
3,160,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
4,159,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,19,False,False,False,False,False,BNK1/2024/,3,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2496,18,False,False,False,False,False,BNK1/2024/,2,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2497,17,False,False,False,False,False,BNK1/2024/,1,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2498,11,False,False,False,False,False,RBILL/2023/05/,1,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False


In [147]:
ii_df['invoice_date']

NameError: name 'ii_df' is not defined

In [148]:
# Filtrar las columnas que contienen 'date' en su nombre
date_columns = [col for col in order.columns if 'date' in col.lower()]

# Crear un nuevo dataframe solo con las columnas que contienen 'date'
date_df = order[date_columns]

# Mostrar el nuevo dataframe filtrado
date_df


,date,due_date,updated_at,expected_payment_date,date_from,date_to,linked_payments_txn_date,issued_credit_notes_cn_date
10478,2024-09-01 00:06:31-03:00,2024-09-01 00:06:31-03:00,2024-09-01 00:06:35-03:00,NaN,2024-09-01,2024-10-01 00:06:31-03:00,1.725160e+09,NaN
10477,2024-09-01 00:06:31-03:00,2024-09-01 00:06:31-03:00,2024-09-01 00:06:35-03:00,NaN,2024-09-01,2024-10-01 00:06:31-03:00,1.725160e+09,NaN
10476,2024-09-01 00:06:31-03:00,2024-09-01 00:06:31-03:00,2024-09-01 00:06:35-03:00,NaN,2024-09-01,2024-10-01 00:06:31-03:00,1.725160e+09,NaN
10475,2024-09-01 00:06:31-03:00,2024-09-01 00:06:31-03:00,2024-09-01 00:06:35-03:00,NaN,2024-09-01,2024-10-01 00:06:31-03:00,1.725160e+09,NaN
9723,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,2024-09-01 00:00:35-03:00,NaN,2024-08-30,2024-08-30 12:11:17-03:00,NaN,NaN
...,...,...,...,...,...,...,...,...
2,2024-08-01 07:17:00-03:00,2024-08-01 07:17:00-03:00,2024-08-13 13:55:55-03:00,NaN,2024-08-01,2024-09-01 07:17:00-03:00,1.723050e+09,NaN
5,2024-08-01 07:17:00-03:00,2024-08-01 07:17:00-03:00,2024-08-13 13:55:55-03:00,NaN,2024-08-01,2024-09-01 07:17:00-03:00,1.723050e+09,NaN
3,2024-08-01 07:17:00-03:00,2024-08-01 07:17:00-03:00,2024-08-13 13:55:55-03:00,NaN,2024-08-01,2024-09-01 07:17:00-03:00,1.723050e+09,NaN
1,2024-08-01 05:08:00-03:00,2024-08-01 05:08:00-03:00,2024-08-23 14:55:19-03:00,NaN,2024-08-01,2024-09-01 05:08:00-03:00,1.724349e+09,NaN


In [149]:
# Verificar si 'partner_id' contiene valores booleanos o vacíos, y reemplazarlos con una lista vacía o un valor adecuado
last_invoices['partner_id'] = last_invoices['partner_id'].apply(lambda x: x if isinstance(x, (list, tuple)) else [None, None])

# Ahora separa la columna partner_id en dos columnas: 'PartnerID' y 'name'
last_invoices[['PartnerID', 'name']] = pd.DataFrame(last_invoices['partner_id'].tolist(), index=last_invoices.index)

# Convertir el campo PartnerID a tipo numérico
last_invoices['PartnerID'] = pd.to_numeric(last_invoices['PartnerID'], errors='coerce')

# Verificar el resultado y crear el DataFrame final con 'id', 'PartnerID' y 'name'
partner_invoices = last_invoices[['id', 'PartnerID', 'name']]

# Mostrar el resultado
partner_invoices


,id,PartnerID,name
21,97870,46869,5a0c58c9e4b031a2c96d319c
22,97869,46868,536109b2e4b0ae650c483374
23,97868,46867,53c42a63e4b08c5d83b878ff
24,97867,46866,5727680de4b06a6e4222f0c8
25,97866,46865,5a71ba83e4b07b95e31fc272
...,...,...,...
2407,95416,44623,5b47de04e4b03c859495a0f7
2408,95415,44622,5cb4d5d0e4b0bd6bed7869fa
2409,95414,44621,5ad08c66e4b01bf0eefaf887
2410,95413,44620,5aa291c2e4b0c65fcc798ce8


In [150]:
#renombro estos campos para que se pueda hacer un merge entre las invoices de Chargebee con estas que son de Odoo
renamed_partner_invoices = partner_invoices.rename(columns={'name': 'customer_id', 'id': 'invoice_id'})
renamed_partner_invoices

,invoice_id,PartnerID,customer_id
21,97870,46869,5a0c58c9e4b031a2c96d319c
22,97869,46868,536109b2e4b0ae650c483374
23,97868,46867,53c42a63e4b08c5d83b878ff
24,97867,46866,5727680de4b06a6e4222f0c8
25,97866,46865,5a71ba83e4b07b95e31fc272
...,...,...,...
2407,95416,44623,5b47de04e4b03c859495a0f7
2408,95415,44622,5cb4d5d0e4b0bd6bed7869fa
2409,95414,44621,5ad08c66e4b01bf0eefaf887
2410,95413,44620,5aa291c2e4b0c65fcc798ce8


In [151]:
#visualizar el campo customer_id del dataframe principal de chargebee que iba ordenado
order['customer_id']

10478    5a0c58c9e4b031a2c96d319c
10477    5a0c58c9e4b031a2c96d319c
10476    5a0c58c9e4b031a2c96d319c
10475    5a0c58c9e4b031a2c96d319c
9723     536109b2e4b0ae650c483374
                   ...           
2        5aa291c2e4b0c65fcc798ce8
5        5aa291c2e4b0c65fcc798ce8
3        5aa291c2e4b0c65fcc798ce8
1        5acdf7d8e4b014cba3b1724b
0        5acdf7d8e4b014cba3b1724b
Name: customer_id, Length: 10479, dtype: object

In [152]:
#unir los dataframes de Odoo y Chargebee para ver a que customer pertnece cada partner y que invoice_id de Odoo tiene ese partner
#last_invoices
lines_join = pd.merge(renamed_partner_invoices, order, on='customer_id', how='inner')
lines_join

,invoice_id_x,PartnerID,customer_id,values,sub_types,dependant_types,invoice_id_y,po_number,subscription_id,recurring,...,line_item_tiers_quantity_used,line_item_tiers_unit_amount,line_item_tiers_object,line_item_tiers_line_item_id,issued_credit_notes_cn_id,issued_credit_notes_cn_reason_code,issued_credit_notes_cn_create_reason_code,issued_credit_notes_cn_date,issued_credit_notes_cn_total,issued_credit_notes_cn_status
0,97870,46869,5a0c58c9e4b031a2c96d319c,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,1mkVvlDQwVxVgL8H4,True,...,3.0,700.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
1,97870,46869,5a0c58c9e4b031a2c96d319c,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,1mkVvlDQwVxVgL8H4,True,...,1.0,0.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
2,97870,46869,5a0c58c9e4b031a2c96d319c,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,1mkVvlDQwVxVgL8H4,True,...,3.0,700.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
3,97870,46869,5a0c58c9e4b031a2c96d319c,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,1mkVvlDQwVxVgL8H4,True,...,1.0,0.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
4,97869,46868,536109b2e4b0ae650c483374,"{'id': '187913', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187913,Argentina - Transferencia Bancaria (Leadaki),16CY4nUBd1nGm7OLS,True,...,5.0,0.0,line_item_tier,li_Azyk2WUN5VwXQ7GuX,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24130,95413,44620,5aa291c2e4b0c65fcc798ce8,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,6or0jU5PxxMNEhuk,True,...,1.0,0.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
24131,95413,44620,5aa291c2e4b0c65fcc798ce8,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,6or0jU5PxxMNEhuk,True,...,3.0,1000.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
24132,95413,44620,5aa291c2e4b0c65fcc798ce8,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,6or0jU5PxxMNEhuk,True,...,3.0,1000.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
24133,95412,44619,5acdf7d8e4b014cba3b1724b,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,169uzAU4TrwKK33PY,True,...,1.0,0.0,line_item_tier,li_6olL2UKBV7cm22we,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
ordered_lines_join = lines_join.sort_values(by='PartnerID', ascending=False)

In [154]:
lines_join_clean = ordered_lines_join.drop_duplicates(['id', 'total'])
lines_join_clean

,invoice_id_x,PartnerID,customer_id,values,sub_types,dependant_types,invoice_id_y,po_number,subscription_id,recurring,...,line_item_tiers_quantity_used,line_item_tiers_unit_amount,line_item_tiers_object,line_item_tiers_line_item_id,issued_credit_notes_cn_id,issued_credit_notes_cn_reason_code,issued_credit_notes_cn_create_reason_code,issued_credit_notes_cn_date,issued_credit_notes_cn_total,issued_credit_notes_cn_status
0,97870,46869,5a0c58c9e4b031a2c96d319c,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,1mkVvlDQwVxVgL8H4,True,...,3.0,700.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
2,97870,46869,5a0c58c9e4b031a2c96d319c,"{'id': '187914', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},187914,Chile,1mkVvlDQwVxVgL8H4,True,...,3.0,700.0,line_item_tier,li_16BdL2UN5XW1GKMPE,NaN,NaN,NaN,NaN,NaN,NaN
751,97869,46868,536109b2e4b0ae650c483374,"{'id': '187913', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187913,Argentina - Transferencia Bancaria (Leadaki),16CY4nUBd1nGm7OLS,True,...,12.0,1000.0,line_item_tier,li_Azyk2WUN5VwXQ7GuX,NaN,NaN,NaN,NaN,NaN,NaN
749,97869,46868,536109b2e4b0ae650c483374,"{'id': '187913', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187913,Argentina - Transferencia Bancaria (Leadaki),16CY4nUBd1nGm7OLS,True,...,12.0,1000.0,line_item_tier,li_Azyk2WUN5VwXQ7GuX,NaN,NaN,NaN,NaN,NaN,NaN
748,97869,46868,536109b2e4b0ae650c483374,"{'id': '187913', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187913,Argentina - Transferencia Bancaria (Leadaki),16CY4nUBd1nGm7OLS,True,...,12.0,1000.0,line_item_tier,li_Azyk2WUN5VwXQ7GuX,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24128,95414,44621,5ad08c66e4b01bf0eefaf887,"{'id': '185394', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185394,Argentina - Transferencia Bancaria,AzZN3tT4OBaWmiMY,True,...,3.0,0.0,line_item_tier,li_AzyjxKUKC78gU1LOR,NaN,NaN,NaN,NaN,NaN,NaN
24129,95413,44620,5aa291c2e4b0c65fcc798ce8,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,6or0jU5PxxMNEhuk,True,...,1.0,0.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
24130,95413,44620,5aa291c2e4b0c65fcc798ce8,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,6or0jU5PxxMNEhuk,True,...,1.0,0.0,line_item_tier,li_Azz6aoUKC1bN81M2q,NaN,NaN,NaN,NaN,NaN,NaN
24133,95412,44619,5acdf7d8e4b014cba3b1724b,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,169uzAU4TrwKK33PY,True,...,1.0,0.0,line_item_tier,li_6olL2UKBV7cm22we,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
lines_join_clean.loc[lines_join_clean['customer_id'] == '6538aee581e02900329122e6']

,invoice_id_x,PartnerID,customer_id,values,sub_types,dependant_types,invoice_id_y,po_number,subscription_id,recurring,...,line_item_tiers_quantity_used,line_item_tiers_unit_amount,line_item_tiers_object,line_item_tiers_line_item_id,issued_credit_notes_cn_id,issued_credit_notes_cn_reason_code,issued_credit_notes_cn_create_reason_code,issued_credit_notes_cn_date,issued_credit_notes_cn_total,issued_credit_notes_cn_status
9796,96664,44756,6538aee581e02900329122e6,"{'id': '185675', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},185675,Mexico,AzZKRtTuUFPtK4J4j,True,...,20.0,0.0,line_item_tier,li_16BdPgUKKNRAs7ZwQ,NaN,NaN,NaN,NaN,NaN,NaN
9767,96664,44756,6538aee581e02900329122e6,"{'id': '185675', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},185675,Mexico,AzZKRtTuUFPtK4J4j,True,...,52.0,1000.0,line_item_tier,li_16BdPgUKKNRAs7ZwQ,NaN,NaN,NaN,NaN,NaN,NaN
9769,96664,44756,6538aee581e02900329122e6,"{'id': '185675', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},185675,Mexico,AzZKRtTuUFPtK4J4j,True,...,52.0,1000.0,line_item_tier,li_16BdPgUKKNRAs7ZwQ,NaN,NaN,NaN,NaN,NaN,NaN
9803,96664,44756,6538aee581e02900329122e6,"{'id': '185675', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},185675,Mexico,AzZKRtTuUFPtK4J4j,True,...,52.0,1000.0,line_item_tier,li_16BdPgUKKNRAs7ZwQ,NaN,NaN,NaN,NaN,NaN,NaN
9795,96664,44756,6538aee581e02900329122e6,"{'id': '185675', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},185675,Mexico,AzZKRtTuUFPtK4J4j,True,...,52.0,1000.0,line_item_tier,li_16BdPgUKKNRAs7ZwQ,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22658,95567,44756,6538aee581e02900329122e6,"{'id': '187818', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},187818,Mexico,None,False,...,NaN,NaN,NaN,NaN,CN-7014,service_unsatisfactory,Service Unsatisfactory,1.726166e+09,14900.0,refunded
22659,95567,44756,6538aee581e02900329122e6,"{'id': '187400', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},187400,Mexico,AzZKRtTuUFPtK4J4j,True,...,1.0,289.0,line_item_tier,li_AzywhLUMPG6NKCbEA,NaN,NaN,NaN,NaN,NaN,NaN
22660,95567,44756,6538aee581e02900329122e6,"{'id': '187307', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},187307,Mexico,AzZKRtTuUFPtK4J4j,True,...,1.0,328.0,line_item_tier,li_169xOfUMIDFap3pX6,NaN,NaN,NaN,NaN,NaN,NaN
22661,95567,44756,6538aee581e02900329122e6,"{'id': '187306', 'po_number': 'Mexico', 'custo...",{'line_items': <class 'chargebee.models.invoic...,{},187306,Mexico,AzZKRtTuUFPtK4J4j,True,...,1.0,328.0,line_item_tier,li_16CGgGUMIDFM73xvy,NaN,NaN,NaN,NaN,NaN,NaN


In [300]:
#corrigiendo datos gracias a un customer con duplicados

#ddaddadaf = lines_join_clean.loc[lines_join_clean['customer_id'] == '6538aee581e02900329122e6']
#ddaddadaf

In [250]:
#ddaddadaf[['customer_id', 'total']]

In [368]:
#aqui me encargo de renombrar campos de el registro de las ultimas invoices insertadas para poderlo unir con otro dataframe mas adelante
last_moves = last_invoices.rename(columns={'id': 'move_id_num'})
last_moves

,move_id_num,partner_id,currency_id,amount_untaxed,amount_tax,PartnerID,name
21,97870,"[46869, 5a0c58c9e4b031a2c96d319c]","[1, USD]",0.0,0.0,46869,5a0c58c9e4b031a2c96d319c
22,97869,"[46868, 536109b2e4b0ae650c483374]","[1, USD]",0.0,0.0,46868,536109b2e4b0ae650c483374
23,97868,"[46867, 53c42a63e4b08c5d83b878ff]","[1, USD]",0.0,0.0,46867,53c42a63e4b08c5d83b878ff
24,97867,"[46866, 5727680de4b06a6e4222f0c8]","[1, USD]",0.0,0.0,46866,5727680de4b06a6e4222f0c8
25,97866,"[46865, 5a71ba83e4b07b95e31fc272]","[1, USD]",0.0,0.0,46865,5a71ba83e4b07b95e31fc272
...,...,...,...,...,...,...,...
2407,95416,"[44623, 5b47de04e4b03c859495a0f7]","[1, USD]",0.0,0.0,44623,5b47de04e4b03c859495a0f7
2408,95415,"[44622, 5cb4d5d0e4b0bd6bed7869fa]","[1, USD]",0.0,0.0,44622,5cb4d5d0e4b0bd6bed7869fa
2409,95414,"[44621, 5ad08c66e4b01bf0eefaf887]","[1, USD]",0.0,0.0,44621,5ad08c66e4b01bf0eefaf887
2410,95413,"[44620, 5aa291c2e4b0c65fcc798ce8]","[1, USD]",0.0,0.0,44620,5aa291c2e4b0c65fcc798ce8


In [302]:
#una busqueda de registro por id


#last_moves.loc[last_moves['move_id_num'] == 20097]

In [303]:
print_column_types(last_invoices)

id 	 int64
partner_id 	 object
currency_id 	 object
amount_untaxed 	 float64
amount_tax 	 float64
PartnerID 	 int64
name 	 object


In [333]:
#order

In [100]:
#a partir del dataframe de Chargebee, me toca aislar todos los campos que incluyen los line items, descuentos y notas de credito

Chargebee_li = order[['id_line_items_expanded', 'date_from', 'date_to', 'unit_amount', 'quantity', 'amount', 'pricing_model', 'is_taxed',
       'tax_amount', 'object_line_items_expanded', 'description', 'entity_type', 'entity_id', 'entity_description', 'tax_exempt_reason'
       , 'discount_amount', 'item_level_discount_amount', 'starting_unit', 'ending_unit', 'quantity_used',
       'unit_amount_line_item_tiers_expanded', 'line_item_id', 'txn_id', 'applied_amount', 'applied_at', 'txn_status', 'txn_date',
       'txn_amount', 'applied_amount_applied_credits_expanded', 'applied_at_applied_credits_expanded', 'cn_id', 'cn_reason_code',
       'cn_create_reason_code', 'cn_date', 'cn_status', 'cn_id_adjustment_credit_notes_expanded',
       'cn_reason_code_adjustment_credit_notes_expanded', 'cn_create_reason_code_adjustment_credit_notes_expanded',
       'cn_date_adjustment_credit_notes_expanded', 'cn_total', 'cn_status_adjustment_credit_notes_expanded',
       'cn_id_issued_credit_notes_expanded', 'cn_reason_code_issued_credit_notes_expanded',
       'cn_create_reason_code_issued_credit_notes_expanded', 'cn_date_issued_credit_notes_expanded',
       'cn_total_issued_credit_notes_expanded', 'cn_status_issued_credit_notes_expanded', 'created_at', 'attempt', 'dunning_type',
       'transaction_id', 'txn_status_dunning_attempts_expanded', 'txn_amount_dunning_attempts_expanded', 'object_discounts_expanded',
       'entity_type_discounts_expanded', 'description_discounts_expanded', 'amount_discounts_expanded', 'discount_type',
       'discount_percentage', 'object_line_item_discounts_expanded', 'line_item_id_line_item_discounts_expanded',
       'discount_type_line_item_discounts_expanded', 'discount_amount_line_item_discounts_expanded', 'coupon_id',
       'entity_id_line_item_discounts_expanded', 'note', 'entity_type_notes_expanded', 'entity_type_notes_expanded']]

In [101]:
cbli = Chargebee_li.drop_duplicates('id_line_items_expanded')
cbli

,id_line_items_expanded,date_from,date_to,unit_amount,quantity,amount,pricing_model,is_taxed,tax_amount,object_line_items_expanded,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_type_notes_expanded
0,li_6olL2UKBV7cg22wd,2024-08-01,2024-09-01 05:08:00-03:00,8900,1,8900,flat_fee,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,li_6olL2UKBV7cm22we,2024-08-01,2024-09-01 05:08:00-03:00,0,1,0,tiered,False,0,line_item,...,NaN,line_item_discount,li_Azz6aoUKC1bN81M2q,item_level_coupon,2000.0,Migra2024Premium2Users,Migra2024Premium2Users,NaN,NaN,NaN
2,li_Azz6aoUKC1bN11M2p,2024-08-01,2024-09-01 07:17:00-03:00,8900,1,8900,flat_fee,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,li_Azz6aoUKC1bN81M2q,2024-08-01,2024-09-01 07:17:00-03:00,750,4,3000,tiered,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,li_AzyjxKUKC78gO1LOQ,2024-08-01,2024-09-01 07:39:00-03:00,39,100,3900,stairstep,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2457,li_AzZnNZULMe4v63Smj,2024-08-13,2024-09-13 17:21:28-03:00,3000,1,3000,flat_fee,False,0,line_item,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2451,li_16BdPRULMXR8j76Yg,2024-08-13,2024-09-13 16:55:04-03:00,48,50,2400,stairstep,False,0,line_item,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2452,li_16BdPRULMXR8o76Yh,2024-08-13,2024-09-13 16:55:04-03:00,0,3,0,tiered,False,0,line_item,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2444,li_16BdPRULMPptF6sd4,2024-08-13,2024-09-13 16:24:52-03:00,467,3,1400,tiered,False,0,line_item,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [304]:
# Obtener la descripción del modelo 'account.move.line'
fields_description = odoo.execute('account.move.line', 'fields_get', [], {'attributes': ['String', 'type']})
lines = pd.DataFrame(fields_description)
lines

,analytic_distribution,analytic_distribution_search,analytic_precision,move_id,journal_id,company_id,company_currency_id,move_name,parent_state,date,...,invoice_origin,payment_matched_amount,l10n_pe_group_id,filter_amount,tax_settlement_move_id,tax_state,subscription_id,subscription_mrr,move_line_ids,withholding_id
type,json,json,integer,many2one,many2one,many2one,many2one,char,selection,date,...,char,monetary,many2one,float,many2one,selection,many2one,monetary,many2many,many2one
searchable,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [306]:
# Mostrar las columnas (campos) del modelo
for field, description in fields_description.items():
    print(f"Campo: {field}, Tipo: {description['type']}, Descripción: {description['string']}")

KeyError: 'string'

In [307]:
# Obtener todos los campos del modelo account.move.line
fields_move_line = odoo.execute('account.move.line', 'fields_get', {})

# Filtrar los campos que contengan 'tax' o 'tx' y extraer solo los nombres de los campos
filtered_fields = [field for field in fields_move_line.keys() if 'tax' in field or 'tx' in field]

# Mostrar los campos filtrados
taxes_df = pd.DataFrame(filtered_fields)
taxes_df

,0
0,tax_ids
1,group_tax_id
2,tax_line_id
3,tax_group_id
4,tax_base_amount
5,tax_repartition_line_id
6,tax_tag_ids
7,tax_tag_invert
8,tax_calculation_rounding_method
9,tax_key


In [70]:
#tax_ids
#group_tax_id
#tax_line_id
#tax_group_id
#tax_base_amount
#tax_repartition_line_id
#tax_tag_ids
#tax_tag_invert
#tax_calculation_rounding_method
#tax_key
#compute_all_tax
#compute_all_tax_dirty
#non_deductible_tax_value
#tax_settlement_move_id
#tax_state

In [ ]:
#ejemplo de creacion de line_item (uno a la vez)

#odoo.create('account.move.line', [{
#    'move_id': 66064,  # ID del movimiento contable (relacionado con account.move)
#    #'account_id': 3,  # ID de la cuenta contable
#    'name': 'Venta de producto',  # Descripción de la línea
#    'quantity': 1.0,  # Cantidad
#    'price_unit': 28.75,  # Precio unitario
#    'debit': 0.0,  # Débito
#    #'credit': 28.75,  # Crédito
#    'partner_id': 34827,  # ID del cliente
#    #'company_id': 1,  # ID de la compañía
#    'currency_id': 1,  # ID de la moneda
#    #'date_maturity': '2024-09-30',  # Fecha de vencimiento
#}])

In [369]:
import signal
import time

# Variable global para indicar si hubo una interrupción manual o del sistema
interrupted = False

# Función para manejar señales de interrupción (como CTRL+C o kill)
def handle_interrupt(signum, frame):
    global interrupted
    print(f"Señal {signum} recibida. Iniciando rollback...")
    interrupted = True  # Marcar interrupción
    raise Exception("Proceso interrumpido manualmente o por el sistema.")

# Conectar las señales de interrupción al manejador de interrupciones
signal.signal(signal.SIGINT, handle_interrupt)   # CTRL+C
signal.signal(signal.SIGTERM, handle_interrupt)  # kill command

def insert_account_move_lines_with_rollback(lines_join_clean, execution_id):
    global interrupted
    records_to_insert = []
    inserted_record_ids = []
    
    # Crear un set para verificar si los move_id ya fueron insertados en este lote
    move_ids_inserted_in_batch = set()

    # Obtener los 'move_id' existentes en account.move.line para evitar duplicados
    existing_move_ids = odoo.search_read('account.move.line', [], ['move_id'])
    
    # Extraer solo el ID de la lista move_id y agregarlo al conjunto
    existing_move_ids_set = set([item['move_id'][0] for item in existing_move_ids if isinstance(item['move_id'], list)])
    
    # Iterar sobre las filas del DataFrame
    for _, row in lines_join_clean.iterrows():
        move_id = row['invoice_id']
        partner_id = row['PartnerID']
        name = row['description']
        quantity = row['quantity']
        amount = row['amount']
        tax = row['tax']

        # Verificar si el move_id ya existe en Odoo o si ya fue procesado en este lote
        if move_id in existing_move_ids_set:
            print(f"move_id {move_id} ya existe en account.move.line. Saltando.")
            continue
        if move_id in move_ids_inserted_in_batch:
            print(f"Duplicado en el lote actual: move_id {move_id}. Saltando.")
            continue

        # Añadir move_id al set de IDs procesados en el lote actual
        move_ids_inserted_in_batch.add(move_id)

        # Crear el registro con los valores a insertar
        record = {
            'move_id': move_id,
            'name': name,  # Descripción de la línea
            'quantity': quantity,  # Cantidad
            'price_unit': amount / 100,  # Precio unitario
           # 'debit': 0.0,  # Débito
            'partner_id': partner_id,  # ID del cliente
            'currency_id': 1,  # ID de la moneda
            'tax_base_amount': tax
        }

        if 'move_id' not in record or not record['move_id']:
            print(f"Registro inválido: {record}")
            continue

        # Agregar el registro a la lista de inserción
        records_to_insert.append(record)

    if records_to_insert:
        try:
            # Realizar las inserciones en un bucle
            for record in records_to_insert:
                if interrupted:
                    raise Exception("Interrupción detectada antes de insertar.")
                
                # Crear un registro en account.move.line
                result = odoo.execute('account.move.line', 'create', [record])
                inserted_record_ids.append(result)

            print(f"Se insertaron {len(inserted_record_ids)} nuevas líneas en account.move.line.")

        except Exception as e:
            print(f"Error durante la inserción: {str(e)}")

            # Intentar eliminar los registros ya insertados en caso de error
            if inserted_record_ids:
                try:
                    print(f"Intentando eliminar los registros insertados debido al error...")
                    odoo.execute('account.move.line', 'unlink', inserted_record_ids)
                    print("Registros eliminados correctamente.")
                except Exception as rollback_error:
                    print(f"Error al intentar eliminar registros: {str(rollback_error)}")

    else:
        print("No se encontraron registros válidos para insertar en account.move.line.")


In [371]:
# Llamada a la función corregida
insert_account_move_lines_with_rollback(lines_join_clean, execution_id='1')


Duplicado en el lote actual: move_id 97635. Saltando.
Duplicado en el lote actual: move_id 97635. Saltando.
Duplicado en el lote actual: move_id 97635. Saltando.
Duplicado en el lote actual: move_id 97821. Saltando.
Duplicado en el lote actual: move_id 97758. Saltando.
Duplicado en el lote actual: move_id 97258. Saltando.
Duplicado en el lote actual: move_id 97763. Saltando.
Duplicado en el lote actual: move_id 97555. Saltando.
Duplicado en el lote actual: move_id 97098. Saltando.
Duplicado en el lote actual: move_id 97621. Saltando.
Duplicado en el lote actual: move_id 97718. Saltando.
Duplicado en el lote actual: move_id 97718. Saltando.
Duplicado en el lote actual: move_id 97718. Saltando.
Duplicado en el lote actual: move_id 97718. Saltando.
Duplicado en el lote actual: move_id 97849. Saltando.
Duplicado en el lote actual: move_id 97780. Saltando.
Duplicado en el lote actual: move_id 97562. Saltando.
Duplicado en el lote actual: move_id 96717. Saltando.
Duplicado en el lote actual:

In [76]:
#tax_ids
#group_tax_id
#tax_line_id
#tax_group_id
#tax_base_amount
#tax_repartition_line_id
#tax_tag_ids
#tax_tag_invert
#tax_calculation_rounding_method
#tax_key
#compute_all_tax
#compute_all_tax_dirty
#non_deductible_tax_value
#tax_settlement_move_id
#tax_state
i7 = odoo.search_read('account.move.line', [], ['id', 'move_id', 'tax_base_amount', 
'compute_all_tax_dirty', 'non_deductible_tax_value', 'tax_settlement_move_id', 'tax_state', 'partner_id', 'price_unit'], limit=None)

# Convertir los resultados a un DataFrame
df_line_items = pd.DataFrame(i7)

# Ordenar el DataFrame por la columna 'id' en orden descendente y seleccionar la cantidad de los ultimos registros que hay que ver
last_li_inv = df_line_items.sort_values(by='id', ascending=False).head(2316)

# Mostrar el DataFrame con los últimos registros mostrando 'id' y 'name'
last_invoices_lines = pd.DataFrame(last_li_inv)
last_invoices_lines

#antes habian 94

{'jsonrpc': '2.0', 'id': 21, 'error': {'code': 200, 'message': 'Odoo Server Error', 'data': {'name': 'psycopg2.OperationalError', 'debug': 'Traceback (most recent call last):\n  File "/home/odoo/src/odoo/odoo/service/model.py", line 133, in retrying\n    result = func()\n  File "/home/odoo/src/odoo/odoo/api.py", line 464, in call_kw\n    result = _call_kw_model(method, model, args, kwargs)\n  File "/home/odoo/src/odoo/odoo/api.py", line 435, in _call_kw_model\n    result = method(recs, *args, **kwargs)\n  File "/home/odoo/src/odoo/odoo/models.py", line 5767, in search_read\n    return records._read_format(fnames=fields, **read_kwargs)\n  File "/home/odoo/src/odoo/odoo/models.py", line 3769, in _read_format\n    vals[name] = convert(record[name], record, use_display_name)\n  File "/home/odoo/src/odoo/odoo/models.py", line 6649, in __getitem__\n    return self._fields[key].__get__(self, self.env.registry[self._name])\n  File "/home/odoo/src/odoo/odoo/fields.py", line 1207, in __get__\n  

Exception: Command has failed

In [92]:
sorted_last_invoices_lines = last_invoices_lines.sort_values(by=['move_id'], ascending=[False])
sorted_last_invoices_lines

,id,move_id,tax_base_amount,compute_all_tax_dirty,non_deductible_tax_value,tax_settlement_move_id,tax_state,partner_id,price_unit
46,92795,"[88351, Draft Invoice (* 88351)]",0.0,True,0.0,False,False,"[44618, 5a0c58c9e4b031a2c96d319c]",0.0
47,92796,"[88351, Draft Invoice (* 88351)]",0.0,True,0.0,False,False,"[44618, 5a0c58c9e4b031a2c96d319c]",0.0
48,92797,"[88350, Draft Invoice (* 88350)]",0.0,True,0.0,False,False,"[44617, 536109b2e4b0ae650c483374]",0.0
49,92798,"[88350, Draft Invoice (* 88350)]",0.0,True,0.0,False,False,"[44617, 536109b2e4b0ae650c483374]",0.0
50,92799,"[88349, Draft Invoice (* 88349)]",0.0,True,0.0,False,False,"[44616, 53c42a63e4b08c5d83b878ff]",129.0
...,...,...,...,...,...,...,...,...,...
4832,97711,"[85893, Draft Invoice (* 85893)]",0.0,True,0.0,False,False,"[42367, 5ad08c66e4b01bf0eefaf887]",89.0
4835,97714,"[85892, Draft Invoice (* 85892)]",0.0,True,0.0,False,False,"[42366, 5aa291c2e4b0c65fcc798ce8]",0.0
4834,97713,"[85892, Draft Invoice (* 85892)]",0.0,True,0.0,False,False,"[42366, 5aa291c2e4b0c65fcc798ce8]",0.0
4836,97715,"[85891, Draft Invoice (* 85891)]",0.0,True,0.0,False,False,"[42365, 5acdf7d8e4b014cba3b1724b]",89.0


In [94]:
unique_sorted_last_invoices_lines = sorted_last_invoices_lines.drop_duplicates('move_id')
unique_sorted_last_invoices_lines

,id,move_id,tax_base_amount,compute_all_tax_dirty,non_deductible_tax_value,tax_settlement_move_id,tax_state,partner_id,price_unit
46,92795,"[88351, Draft Invoice (* 88351)]",0.0,True,0.0,False,False,"[44618, 5a0c58c9e4b031a2c96d319c]",0.00
48,92797,"[88350, Draft Invoice (* 88350)]",0.0,True,0.0,False,False,"[44617, 536109b2e4b0ae650c483374]",0.00
50,92799,"[88349, Draft Invoice (* 88349)]",0.0,True,0.0,False,False,"[44616, 53c42a63e4b08c5d83b878ff]",129.00
52,92801,"[88348, Draft Invoice (* 88348)]",0.0,True,0.0,False,False,"[44615, 5727680de4b06a6e4222f0c8]",9.01
54,92803,"[88347, Draft Invoice (* 88347)]",0.0,True,0.0,False,False,"[44614, 5a71ba83e4b07b95e31fc272]",500.00
...,...,...,...,...,...,...,...,...,...
4829,97708,"[85895, Draft Invoice (* 85895)]",0.0,True,0.0,False,False,"[42369, 5b47de04e4b03c859495a0f7]",0.00
4831,97710,"[85894, Draft Invoice (* 85894)]",0.0,True,0.0,False,False,"[42368, 5cb4d5d0e4b0bd6bed7869fa]",0.00
4833,97712,"[85893, Draft Invoice (* 85893)]",0.0,True,0.0,False,False,"[42367, 5ad08c66e4b01bf0eefaf887]",0.00
4835,97714,"[85892, Draft Invoice (* 85892)]",0.0,True,0.0,False,False,"[42366, 5aa291c2e4b0c65fcc798ce8]",0.00


In [293]:
last_invoices_lines.loc[last_invoices_lines['tax_settlement_move_id'] != False]

,id,move_id,tax_base_amount,compute_all_tax_dirty,non_deductible_tax_value,tax_settlement_move_id,tax_state


In [344]:
#mas adelante intente hacer una eliminacion de muchos line_items pero resulta que al ingresar una nueva invoice, se tiene que eliminar 
# el asiento contable para que se pueda eliminar tanto la invoice como sus line_items
# asi que primero decido separar el campo move_id ya que es un one2many y necesito convertirlo a numerico para poderlo eliminar

# Separar la columna move_id en dos columnas: move_id (numérico) y description
last_invoices_lines[['move_id_num', 'description']] = pd.DataFrame(last_invoices_lines['move_id'].tolist(), index=last_invoices_lines.index)

# Convertir el campo move_id_num a tipo numérico
last_invoices_lines['move_id_num'] = pd.to_numeric(last_invoices_lines['move_id_num'], errors='coerce')

# Verificar el resultado
last_invoices_lines_refined = last_invoices_lines[['id', 'move_id_num', 'description']]
last_invoices_lines_refined

,id,move_id_num,description
4423,103792,92953,Draft Invoice (* 92953)
4422,103791,92953,Draft Invoice (* 92953)
4421,103790,92954,Draft Invoice (* 92954)
4420,103789,92954,Draft Invoice (* 92954)
4419,103788,92955,Draft Invoice (* 92955)
...,...,...,...
50,99299,95409,Draft Invoice (* 95409)
49,99298,95410,Draft Invoice (* 95410)
48,99297,95410,Draft Invoice (* 95410)
47,99296,95411,Draft Invoice (* 95411)


In [345]:
#last_moves que genere antes ahora lo uno con el dataframe que me permite separar el move_id de su numero y descripcion en el modelo
#account.move.line porque ese id lo muestra con un array tipo asi [45268, 'Draft Invoice (* 45268)']
lines_i_join = pd.merge(last_moves, last_invoices_lines_refined, on='move_id_num', how='inner')
lines_i_join

,move_id_num,partner_id,currency_id,amount_untaxed,amount_tax,PartnerID,name,id,description
0,95411,"[46869, 5a0c58c9e4b031a2c96d319c]","[1, USD]",0.0,0.0,46869,5a0c58c9e4b031a2c96d319c,99296,Draft Invoice (* 95411)
1,95411,"[46869, 5a0c58c9e4b031a2c96d319c]","[1, USD]",0.0,0.0,46869,5a0c58c9e4b031a2c96d319c,99295,Draft Invoice (* 95411)
2,95410,"[46868, 536109b2e4b0ae650c483374]","[1, USD]",0.0,0.0,46868,536109b2e4b0ae650c483374,99298,Draft Invoice (* 95410)
3,95410,"[46868, 536109b2e4b0ae650c483374]","[1, USD]",0.0,0.0,46868,536109b2e4b0ae650c483374,99297,Draft Invoice (* 95410)
4,95409,"[46867, 53c42a63e4b08c5d83b878ff]","[1, USD]",0.0,0.0,46867,53c42a63e4b08c5d83b878ff,99300,Draft Invoice (* 95409)
...,...,...,...,...,...,...,...,...,...
4493,92955,"[44621, 5ad08c66e4b01bf0eefaf887]","[1, USD]",0.0,0.0,44621,5ad08c66e4b01bf0eefaf887,103787,Draft Invoice (* 92955)
4494,92954,"[44620, 5aa291c2e4b0c65fcc798ce8]","[1, USD]",0.0,0.0,44620,5aa291c2e4b0c65fcc798ce8,103790,Draft Invoice (* 92954)
4495,92954,"[44620, 5aa291c2e4b0c65fcc798ce8]","[1, USD]",0.0,0.0,44620,5aa291c2e4b0c65fcc798ce8,103789,Draft Invoice (* 92954)
4496,92953,"[44619, 5acdf7d8e4b014cba3b1724b]","[1, USD]",0.0,0.0,44619,5acdf7d8e4b014cba3b1724b,103792,Draft Invoice (* 92953)


In [350]:
#buscar un registro en account.move.line por medio de su id

#lines_i_join.loc[lines_i_join['move_id_num']==83370]

In [346]:
# Reversar los movimientos contables
move_ids = last_invoices_lines['move_id_num'].dropna().astype(int).tolist()  # Convertir a enteros y eliminar valores NaN
print("IDs de movimientos a procesar:", move_ids)


IDs de movimientos a procesar: [92953, 92953, 92954, 92954, 92955, 92955, 92956, 92956, 92957, 92957, 92958, 92958, 92959, 92959, 92960, 92960, 92962, 92962, 92963, 92963, 92964, 92964, 92965, 92965, 92966, 92966, 92967, 92967, 92968, 92968, 92969, 92969, 92970, 92970, 92971, 92971, 92972, 92972, 92973, 92973, 92974, 92974, 92976, 92976, 92977, 92977, 92978, 92978, 92979, 92979, 92980, 92980, 92981, 92981, 92982, 92982, 92983, 92983, 92984, 92984, 92985, 92985, 92986, 92986, 92987, 92987, 92988, 92988, 92989, 92989, 92990, 92990, 92991, 92991, 92992, 92992, 92993, 92993, 92994, 92994, 92996, 92996, 92997, 92997, 92998, 92998, 92999, 92999, 93001, 93001, 93002, 93002, 93003, 93003, 93004, 93004, 93005, 93005, 93006, 93006, 93007, 93007, 93008, 93008, 93009, 93009, 93010, 93010, 93011, 93011, 93013, 93013, 93014, 93014, 93015, 93015, 93016, 93016, 93017, 93017, 93018, 93018, 93019, 93019, 93020, 93020, 93021, 93021, 93022, 93022, 93023, 93023, 93024, 93024, 93025, 93025, 93026, 93026, 93

In [348]:
# Reversar los movimientos contables osea eliminar todos los registros contables, lo que consiste en eliminar los line items de cada
# invoice y a la vez tambien se elimine el registro padre que es el de account.move, pero esta manera de liminarlos fue la que pude
#utilizar :(
move_ids = last_invoices_lines_refined['move_id_num'].dropna().astype(int).tolist()  # Convertir a enteros y eliminar valores NaN

# Usar un conjunto para asegurarse de no procesar IDs duplicados
processed_ids = set()

if move_ids:
    try:
        for move_id in move_ids:
            # Verificar si el ID ya ha sido procesado
            if move_id in processed_ids:
                print(f"ID de movimiento {move_id} ya fue procesado, saltando...")
                continue
            
            print(f"Procesando ID de movimiento: {move_id} (Tipo: {type(move_id)})")
            
            # Verificar si el movimiento existe
            move_record = odoo.execute('account.move', 'search_read', [[('id', '=', move_id)]], {'fields': ['state']})
            
            if move_record:
                if move_record[0]['state'] != 'draft':
                    # Si el estado no es 'draft', cambiarlo a 'draft'
                    odoo.execute('account.move', 'button_draft', [[move_id]])
                    print(f"Asiento contable {move_id} puesto en estado borrador.")
                
                # Eliminar el movimiento contable
                odoo.execute('account.move', 'unlink', [[move_id]])
                print(f"Asiento contable {move_id} eliminado.")
                
                # Marcar el ID como procesado
                processed_ids.add(move_id)
            else:
                # Si no encuentra el movimiento, continuar con el siguiente
                print(f"Asiento contable {move_id} no encontrado, posiblemente ya ha sido eliminado previamente.")
                processed_ids.add(move_id)  # Marcarlo como procesado para evitar reintentos
                continue  # Sigue con el siguiente movimiento

        print(f"Se han revertido y eliminado {len(processed_ids)} asientos contables.")
        
    except Exception as e:
        print(f"Error al revertir/eliminar los asientos: {str(e)}")
else:
    print("No se encontraron IDs válidos para revertir/eliminar.")


Procesando ID de movimiento: 92953 (Tipo: <class 'int'>)
Asiento contable 92953 eliminado.
ID de movimiento 92953 ya fue procesado, saltando...
Procesando ID de movimiento: 92954 (Tipo: <class 'int'>)
Asiento contable 92954 eliminado.
ID de movimiento 92954 ya fue procesado, saltando...
Procesando ID de movimiento: 92955 (Tipo: <class 'int'>)
Asiento contable 92955 eliminado.
ID de movimiento 92955 ya fue procesado, saltando...
Procesando ID de movimiento: 92956 (Tipo: <class 'int'>)
Asiento contable 92956 eliminado.
ID de movimiento 92956 ya fue procesado, saltando...
Procesando ID de movimiento: 92957 (Tipo: <class 'int'>)
Asiento contable 92957 eliminado.
ID de movimiento 92957 ya fue procesado, saltando...
Procesando ID de movimiento: 92958 (Tipo: <class 'int'>)
Asiento contable 92958 eliminado.
ID de movimiento 92958 ya fue procesado, saltando...
Procesando ID de movimiento: 92959 (Tipo: <class 'int'>)
Asiento contable 92959 eliminado.
ID de movimiento 92959 ya fue procesado, salt

In [ ]:
#habian 94 registros

In [107]:
#cbli.loc[(cbli['cn_id'].notna()) & (cbli['cn_id'].notna())]

In [112]:
cbli.loc[cbli['cn_id'].notna()]

,id_line_items_expanded,date_from,date_to,unit_amount,quantity,amount,pricing_model,is_taxed,tax_amount,object_line_items_expanded,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_type_notes_expanded
22,li_Azyk9vUKCbLnIEIQ,2024-08-01,2024-09-01 09:39:00-03:00,0,1,0,tiered,False,0,line_item,...,NaN,line_item_discount,li_169mLZUKCkROfJfI,item_level_coupon,4340.0,2024[ESCORPO]10%FOREVER,2024[ESCORPO]10%FOREVER,NaN,NaN,NaN
111,li_16BdHGUKDFsgM2jJ6,2024-08-01,2024-09-01 12:20:00-03:00,0,1,0,flat_fee,False,0,line_item,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,li_16BdHGUKDZlfT3m2p,2024-08-01,2024-09-01 13:39:00-03:00,33,400,13000,stairstep,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*Paquete de HSM 16000 USD 978/ ...,customer,customer
156,li_16BdHGUKDZlfY3m2q,2024-08-01,2024-09-01 13:39:00-03:00,0,1,0,flat_fee,False,0,line_item,...,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,li_16BdHGUKDZlfZ3m2r,2024-08-01,2024-09-01 13:39:00-03:00,0,6,0,tiered,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2229,li_16BdGRULFHo7lMc4C,2024-08-12,2024-09-12 11:09:05-03:00,3900,1,3900,flat_fee,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2297,li_16BdGRULGVQFvPNwC,2024-08-12,2024-09-12 16:09:26-03:00,0,1,0,tiered,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2307,li_Azz6neULGd4In5LSi,2024-08-12,2024-09-12 16:39:48-03:00,9900,1,9900,flat_fee,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2395,li_16BdPRULLKygg4YsZ,2024-08-13,2024-09-13 11:59:18-03:00,0,3,0,tiered,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
i10 = odoo.search_read('account.move.reversal', [], [], limit=None)

# Convertir los resultados a un DataFrame
df_credit_notes = pd.DataFrame(i10)

# Ordenar el DataFrame por la columna 'id' en orden descendente y seleccionar la cantidad de los ultimos registros que hay que ver
last_li_cn = df_credit_notes.sort_values(by='id', ascending=False).head(27)

# Mostrar el DataFrame con los últimos registros mostrando 'id' y 'name'
last_credit_notes = pd.DataFrame(last_li_cn)
last_credit_notes

,id,move_ids,new_move_ids,date,reason,journal_id,company_id,available_journal_ids,country_code,residual,...,l10n_latam_use_documents,l10n_latam_document_type_id,l10n_latam_available_document_type_ids,l10n_latam_document_number,l10n_latam_manual_document_number,l10n_es_edi_facturae_reason_code,l10n_cl_edi_reference_doc_code,l10n_cl_is_text_correction,l10n_cl_original_text,l10n_cl_corrected_text
5,6,[],[],2024-08-01,demo_sup_refund_invoice_5: liquido producto bi...,"[42, Vendor Bills]","[6, Muebleria ARG]",[],AR,0.0,...,False,"[142, (186) DIRECT PURCHASE LIQUIDATION A - LI...",[],00011-00000012,False,10,False,False,False,False
4,5,[],[],2024-08-15,Venta cancelada,"[42, Vendor Bills]","[6, Muebleria ARG]",[],AR,0.0,...,False,"[119, FOREIGN CREDIT NOTES AND REIMBURSEMENTS]",[],0001-01234566,False,10,False,False,False,False
3,4,[],[],2024-08-15,Mercadería defectuosa,"[42, Vendor Bills]","[6, Muebleria ARG]",[],AR,0.0,...,False,"[119, FOREIGN CREDIT NOTES AND REIMBURSEMENTS]",[],0001-01234567,False,10,False,False,False,False
2,3,[],[],2024-08-01,Venta cancelada,"[51, Expo Sales Journal]","[6, Muebleria ARG]",[],AR,0.0,...,False,"[19, (21) CREDIT NOTES FOR FOREIGN OPERATIONS]",[],False,False,10,False,False,False,False
1,2,[],[],2024-08-01,Venta cancelada,"[41, Ventas Preimpreso]","[6, Muebleria ARG]",[],AR,0.0,...,False,"[3, (3) CREDIT NOTES A]",[],False,False,10,False,False,False,False
0,1,[],[],2024-08-01,Mercadería defectuosa,"[41, Ventas Preimpreso]","[6, Muebleria ARG]",[],AR,0.0,...,False,"[3, (3) CREDIT NOTES A]",[],False,False,10,False,False,False,False


In [89]:
# Simulación de los impuestos disponibles en Odoo (ejemplo)


def create_move(odoo):
    """
    Esta función simula la creación de un movimiento contable (account.move).
    """
    move_vals = {
        'journal_id': 1,  # ID del diario contable
        'date': '2024-10-10',  # Fecha del movimiento
        'ref': 'Referencia de prueba',  # Referencia del movimiento
    }
    move_id = odoo.create('account.move', [move_vals])  # Crear un nuevo account.move
    return move_id

def get_aml_vals(product_id, partner_id, price, quantity):
    """
    Función que retorna los valores de una línea de 'account.move.line'.
    """
    return {
        'product_id': product_id,  # ID del producto
        'partner_id': partner_id,  # ID del cliente
        'name': 'Venta de producto',  # Descripción
        'quantity': quantity,  # Cantidad
        'price_unit': price,  # Precio unitario
        'debit': 0.0,  # Débito (en caso de ser necesario)
        'credit': price * quantity,  # Crédito
        'currency_id': 1,  # ID de la moneda # Impuesto aplicado
    }

def insert_lines_in_batch(odoo, products, partner_id):
    """
    Función que inserta líneas de account.move.line en batch para un account.move.
    
    - products: Lista de productos con su información.
    - partner_id: ID del cliente/partner para asociar a las líneas.
    """
    # 1. Crear el movimiento contable (account.move)
    move_id = create_move(odoo)

    # 2. Preparar las líneas de account.move.line
    aml_vals = []
    
    for product in products:
        product_id = product['product_id']
        price = product['price']
        quantity = product['quantity']
        
        # Generar los valores de la línea
        aml_vals.append(get_aml_vals(product_id, partner_id, price, quantity))
    
    # 3. Insertar las líneas de account.move.line usando create
    try:
        for aml in aml_vals:
            aml['move_id'] = move_id  # Asignar el ID del movimiento contable
            odoo.create('account.move.line', [aml])  # Crear las líneas individualmente
        print(f"Líneas insertadas correctamente en el movimiento con ID {move_id}.")
    except Exception as e:
        print("Error al insertar líneas:", str(e))

# Ejemplo de uso:
products = [
    {'product_id': 101, 'price': 100.0, 'quantity': 1},  # Producto 1
    {'product_id': 102, 'price': 200.0, 'quantity': 2},  # Producto 2
]

partner_id = 34827  # ID del cliente

# Insertar en batch
insert_lines_in_batch(odoo, products, partner_id)


{'jsonrpc': '2.0', 'id': 60, 'error': {'code': 200, 'message': 'Odoo Server Error', 'data': {'name': 'odoo.exceptions.UserError', 'debug': 'Traceback (most recent call last):\n  File "/home/odoo/src/odoo/odoo/http.py", line 1781, in _serve_db\n    return service_model.retrying(self._serve_ir_http, self.env)\n  File "/home/odoo/src/odoo/odoo/service/model.py", line 133, in retrying\n    result = func()\n  File "/home/odoo/src/odoo/odoo/http.py", line 1808, in _serve_ir_http\n    response = self.dispatcher.dispatch(rule.endpoint, args)\n  File "/home/odoo/src/odoo/odoo/http.py", line 2012, in dispatch\n    result = self.request.registry[\'ir.http\']._dispatch(endpoint)\n  File "/home/odoo/src/odoo/addons/website/models/ir_http.py", line 235, in _dispatch\n    response = super()._dispatch(endpoint)\n  File "/home/odoo/src/odoo/odoo/addons/base/models/ir_http.py", line 226, in _dispatch\n    result = endpoint(**request.params)\n  File "/home/odoo/src/odoo/odoo/http.py", line 757, in route_

In [84]:
def insert_lines_in_batch(odoo, products, partner_id):
    """
    Función que inserta líneas de account.move.line en batch para un account.move.
    
    - products: Lista de productos con su información.
    - partner_id: ID del cliente/partner para asociar a las líneas.
    """
    # 1. Crear el movimiento contable (account.move)
    move_id = create_move(odoo)

    # 2. Preparar las líneas de account.move.line
    aml_vals = []
    
    total_debit = 0  # Acumular total de débitos
    for product in products:
        product_id = product['product_id']
        price = product['price']
        quantity = product['quantity']
        
        # Generar los valores de la línea sin impuestos
        aml_vals.append(get_aml_vals(product_id, partner_id, price, quantity))
        total_debit += price * quantity  # Sumar al total de débitos

    # Agregar una línea de crédito para equilibrar
    aml_vals.append({
        'product_id': False,  # Sin producto
        'partner_id': partner_id,  # ID del cliente
        'name': 'Contrapartida',  # Descripción
        'quantity': 1,  # Cantidad de 1 para el crédito
        'price_unit': total_debit,  # Usar el total de débitos como precio unitario
        'debit': 0.0,  # Sin débito
        'credit': total_debit,  # Total de créditos
        'currency_id': 1,  # ID de la moneda
    })

    # Verificar si los totales de débitos y créditos están balanceados
    total_credit = sum(aml['credit'] for aml in aml_vals)
    if total_debit != total_credit:
        print("Error: Los totales de débitos y créditos no están balanceados.")
        return

    # 3. Insertar las líneas de account.move.line usando create
    try:
        for aml in aml_vals:
            aml['move_id'] = move_id  # Asignar el ID del movimiento contable
            odoo.create('account.move.line', [aml])  # Crear las líneas individualmente
        print(f"Líneas insertadas correctamente en el movimiento con ID {move_id}.")
    except Exception as e:
        print("Error al insertar líneas:", str(e))

# Ejemplo de uso:
products = [
    {'product_id': 101, 'price': 100.0, 'quantity': 1},  # Producto 1
    {'product_id': 102, 'price': 200.0, 'quantity': 2},  # Producto 2
]

partner_id = 34827  # ID del cliente

# Insertar en batch
insert_lines_in_batch(odoo, products, partner_id)


{'jsonrpc': '2.0', 'id': 54, 'error': {'code': 200, 'message': 'Odoo Server Error', 'data': {'name': 'odoo.exceptions.UserError', 'debug': 'Traceback (most recent call last):\n  File "/home/odoo/src/odoo/odoo/http.py", line 1781, in _serve_db\n    return service_model.retrying(self._serve_ir_http, self.env)\n  File "/home/odoo/src/odoo/odoo/service/model.py", line 133, in retrying\n    result = func()\n  File "/home/odoo/src/odoo/odoo/http.py", line 1808, in _serve_ir_http\n    response = self.dispatcher.dispatch(rule.endpoint, args)\n  File "/home/odoo/src/odoo/odoo/http.py", line 2012, in dispatch\n    result = self.request.registry[\'ir.http\']._dispatch(endpoint)\n  File "/home/odoo/src/odoo/addons/website/models/ir_http.py", line 235, in _dispatch\n    response = super()._dispatch(endpoint)\n  File "/home/odoo/src/odoo/odoo/addons/base/models/ir_http.py", line 226, in _dispatch\n    result = endpoint(**request.params)\n  File "/home/odoo/src/odoo/odoo/http.py", line 757, in route_

In [75]:
# Búsqueda de productos en el modelo 'product.product'
product_search = odoo.search_read('product.product', [], ['id', 'name'], limit=None)

# Convertir los resultados a un DataFrame para una mejor visualización
products_df = pd.DataFrame(product_search)

# Mostrar los productos encontrados
products_df

,id,name
0,94,Ad-Valorem
1,88,Tariff
2,86,Import Clearance
3,89,Guard Service
4,87,Statistics Rate
...,...,...
104,112,Senior Architect (Invoice on Timesheets)
105,110,Service on Timesheets
106,37,Top-up eWallet
107,4,Virtual Home Staging


In [153]:
products_df.loc[products_df['id'] == 118]

,id,name
73,118,Producto de ejemplo


In [70]:
# Datos del nuevo producto a insertar
new_product_data = {
    'name': 'Producto de ejemplo',
    'type': 'consu',  # tipo de producto ('consu' = Consumible, 'service' = Servicio, 'product' = Almacenable)
    'list_price': 100.00,  # Precio de venta
    'standard_price': 70.00,  # Precio de coste
    'default_code': 'PROD_EX',  # Código interno del producto
    'sale_ok': True,  # Habilitar para ventas
    'purchase_ok': True  # Habilitar para compras
}

# Insertar el nuevo producto usando el método Odoo.create
try:
    new_product_id = odoo.create('product.product', [new_product_data])
    print(f"Producto insertado con éxito. ID: {new_product_id}")
except Exception as e:
    print(f"Error durante la inserción del producto: {str(e)}")


Producto insertado con éxito. ID: 118


In [120]:
# Definir el ID de producto a buscar
product_id_to_delete = 118

# Búsqueda de productos en el modelo 'stock.move'
inventory = odoo.search_read('stock.move', [], ['product_id'], limit=None)

# Convertir los resultados a un DataFrame para una mejor visualización
inventory_df = pd.DataFrame(inventory)

# Extraer el ID del campo 'product_id', que es una lista [ID, Nombre]
# Crear una nueva columna en el DataFrame que contenga solo el número de product_id
inventory_df['product_id_number'] = inventory_df['product_id'].apply(lambda x: x[0] if isinstance(x, list) else None)

# Buscar el registro que tenga el product_id igual a 118
result = inventory_df.loc[inventory_df['product_id_number'] == product_id_to_delete]

# Mostrar los resultados encontrados
if not result.empty:
    print(f"Se encontró el producto con ID {product_id_to_delete} en 'el modelo':\n")
    print(result)
else:
    print(f"No se encontró el producto con ID {product_id_to_delete} en 'el modelo'.")


No se encontró el producto con ID 118 en 'el modelo'.


In [66]:
#hacer el llamado a res.users para mas adelante poder insertar los line items
# Especificar los campos que deseas leer
fields_to_read = ['id', 'name']

# Realizar el llamado al modelo account.move.line con un límite de registros
users = odoo.search_read('res.users', [], fields_to_read, limit=1000)

# Convertir los resultados a un DataFrame
u_df = pd.DataFrame(users)
u_df

,id,name
0,15,Chargebee
1,16,Jim Galvez
2,7,Joel Willis
3,6,Marc Demo
4,10,Valentino
5,11,Valentino
6,14,Valentino
7,13,Valentino
8,12,Valentino


In [119]:
#para no estorbar tanto en las lineas de arriba, hasta qui abajo genere como los queries para los dataframes en los modelos utilizados

#order.select_dtypes(include=['int64', 'float64'])

In [120]:
#muestra todos los campos de tipo object ya que entre ellos estan esos campos a desanidar
order.select_dtypes(include=['int64'])


,net_term_days,total,amount_paid,amount_adjusted,write_off_amount,credits_applied,amount_due,resource_version,amount_to_collect,round_off_amount,tax,sub_total,unit_amount,quantity,amount,tax_amount,discount_amount,item_level_discount_amount
0,0,8900,8900,0,0,0,0,1724435719786,0,0,0,8900,8900,1,8900,0,0,0
1,0,9900,9900,0,0,0,0,1723568155857,0,0,0,9900,0,1,0,0,0,0
2,0,3900,3900,0,0,0,0,1722508742063,0,0,0,3900,8900,1,8900,0,0,0
3,0,3900,3900,0,0,0,0,1723218079989,0,0,0,3900,750,4,3000,0,2000,2000
4,0,23900,23900,0,0,0,0,1722510311847,0,0,0,23900,39,100,3900,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2457,0,0,0,0,0,0,0,1725159618918,0,0,0,0,3000,1,3000,0,0,0
2451,0,0,0,0,0,0,0,1725159619104,0,0,0,0,48,50,2400,0,0,0
2452,0,0,0,0,0,0,0,1725159619316,0,0,0,0,0,3,0,0,0,0
2444,0,0,0,0,0,0,0,1725159635973,0,0,0,0,467,3,1400,0,1400,1400


In [316]:
customerorder1 = order.loc[order['customer_id'] == '6538aee581e02900329122e6']
customerorder1[['id','customer_id', 'total']]

,id,customer_id,total
155,185552,6538aee581e02900329122e6,16466
275,185675,6538aee581e02900329122e6,686100
1223,186658,6538aee581e02900329122e6,606
1224,186659,6538aee581e02900329122e6,606
1226,186661,6538aee581e02900329122e6,605
1227,186662,6538aee581e02900329122e6,605
1228,186663,6538aee581e02900329122e6,605
1229,186664,6538aee581e02900329122e6,605
1230,186665,6538aee581e02900329122e6,605
1232,186667,6538aee581e02900329122e6,604


KeyError: 'total'

In [296]:
lines

,analytic_distribution,analytic_distribution_search,analytic_precision,move_id,journal_id,company_id,company_currency_id,move_name,parent_state,date,...,invoice_origin,payment_matched_amount,l10n_pe_group_id,filter_amount,tax_settlement_move_id,tax_state,subscription_id,subscription_mrr,move_line_ids,withholding_id
string,Analytic Distribution,Analytic Distribution Search,Analytic Precision,Journal Entry,Journal,Company,Company Currency,Number,Status,Date,...,Origin,Payment Matched Amount,Group,Filter Amount,Tax Settlement Move,Tax State,Subscription,Monthly Recurring Revenue,Related Stock Moves,Withholding
type,json,json,integer,many2one,many2one,many2one,many2one,char,selection,date,...,char,monetary,many2one,float,many2one,selection,many2one,monetary,many2many,many2one
searchable,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
oo = order.loc[order['customer_id'] == '6538aee581e02900329122e6']
oo

,id,po_number,customer_id,subscription_id,recurring,status,price_type,date,due_date,net_term_days,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
155,185552,Mexico,6538aee581e02900329122e6,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-01 17:19:07-03:00,2024-08-01 17:19:07-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
275,185675,Mexico,6538aee581e02900329122e6,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-02 17:33:55-03:00,2024-08-02 17:33:55-03:00,0,...,NaN,line_item_discount,li_16BdPgUKJz9CJ6lkn,item_level_coupon,19000.0,LEADAKI100%,LEADAKI100%,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
1222,186658,Mexico,6538aee581e02900329122e6,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 22:55:09-03:00,2024-08-14 22:55:09-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1223,186659,Mexico,6538aee581e02900329122e6,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 22:59:53-03:00,2024-08-14 22:59:53-03:00,0,...,NaN,line_item_discount,li_Azz6ewULS8wFn1SYu,item_level_coupon,700.0,MIGRA2024STARTER1USER,MIGRA2024STARTER1USER,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
1225,186661,Mexico,6538aee581e02900329122e6,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 23:07:08-03:00,2024-08-14 23:07:08-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1226,186662,Mexico,6538aee581e02900329122e6,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 23:11:58-03:00,2024-08-14 23:11:58-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
1227,186663,Mexico,6538aee581e02900329122e6,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 23:20:18-03:00,2024-08-14 23:20:18-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
1228,186664,Mexico,6538aee581e02900329122e6,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 23:28:43-03:00,2024-08-14 23:28:43-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
1229,186665,Mexico,6538aee581e02900329122e6,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 23:45:33-03:00,2024-08-14 23:45:33-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
1231,186667,Mexico,6538aee581e02900329122e6,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-15 00:09:58-03:00,2024-08-15 00:09:58-03:00,0,...,NaN,line_item_discount,li_16BdC4ULSSprV83UH,item_level_coupon,700.0,MIGRA24A23STARTER1AGENTEX3MESES,MIGRA24A23STARTER1AGENTEX3MESES,NaN,NaN,NaN


In [151]:
oo[['customer_id', 'amount', 'quantity', 'total']]

,customer_id,amount,quantity,total
155,6538aee581e02900329122e6,13000,400,16466
275,6538aee581e02900329122e6,0,6,686100
1222,6538aee581e02900329122e6,13000,400,606
1223,6538aee581e02900329122e6,2000,1,606
1225,6538aee581e02900329122e6,700,2,605
1226,6538aee581e02900329122e6,3900,1,605
1227,6538aee581e02900329122e6,0,1,605
1228,6538aee581e02900329122e6,2400,50,605
1229,6538aee581e02900329122e6,0,3,605
1231,6538aee581e02900329122e6,3000,4,604


In [152]:
#muestra todos los campos de tipo object ya que entre ellos estan esos campos a desanidar
order_floats = order.select_dtypes(include=['float64'])
order_floats

,exchange_rate,new_sales_amount,expected_payment_date,next_retry_at,starting_unit,ending_unit,quantity_used,unit_amount_line_item_tiers_expanded,applied_amount,txn_date,...,cn_date_adjustment_credit_notes_expanded,cn_total,cn_date_issued_credit_notes_expanded,cn_total_issued_credit_notes_expanded,created_at,attempt,txn_amount_dunning_attempts_expanded,amount_discounts_expanded,discount_percentage,discount_amount_line_item_discounts_expanded
0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,0.0,8900.0,1.724349e+09,...,NaN,NaN,NaN,NaN,1.722568e+09,1.0,NaN,NaN,NaN,NaN
1,1.0,NaN,NaN,NaN,1.0,1.0,1.0,0.0,9900.0,1.723050e+09,...,NaN,NaN,NaN,NaN,1.723518e+09,2.0,NaN,2000.0,NaN,2000.0
2,1.0,NaN,NaN,NaN,2.0,NaN,3.0,1000.0,3900.0,1.721012e+09,...,NaN,NaN,NaN,NaN,1.724382e+09,3.0,NaN,NaN,NaN,NaN
3,1.0,NaN,NaN,NaN,1.0,3.0,3.0,0.0,3900.0,1.723045e+09,...,NaN,NaN,NaN,NaN,1.722568e+09,1.0,NaN,NaN,NaN,NaN
4,1.0,NaN,NaN,NaN,1.0,1.0,1.0,0.0,23900.0,1.722510e+09,...,NaN,NaN,NaN,NaN,1.723518e+09,2.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,1.0,NaN,NaN,NaN,1.0,1.0,1.0,0.0,9000.0,1.724288e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8900.0,100.0,NaN
2454,1.0,NaN,NaN,NaN,1.0,1.0,1.0,0.0,3900.0,1.724282e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9900.0,100.0,NaN
2455,1.0,NaN,NaN,NaN,2.0,NaN,1.0,700.0,3900.0,1.724283e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3900.0,100.0,NaN
2447,1.0,NaN,NaN,NaN,1.0,6.0,6.0,0.0,3900.0,1.724276e+09,...,NaN,NaN,NaN,NaN,1.724345e+09,0.0,3400.0,7400.0,100.0,NaN


In [318]:
ono = lines_join_clean.loc[lines_join_clean['customer_id'] == '6538aee581e02900329122e6']
ono

,invoice_id,PartnerID,customer_id,id,po_number,subscription_id,recurring,status,price_type,date,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
61,95371,44756,6538aee581e02900329122e6,185552,Mexico,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-01 17:19:07-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
62,95371,44756,6538aee581e02900329122e6,185675,Mexico,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-02 17:33:55-03:00,...,NaN,line_item_discount,li_16BdPgUKJz9CJ6lkn,item_level_coupon,19000.0,LEADAKI100%,LEADAKI100%,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
63,95371,44756,6538aee581e02900329122e6,186658,Mexico,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 22:55:09-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,95371,44756,6538aee581e02900329122e6,186659,Mexico,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 22:59:53-03:00,...,NaN,line_item_discount,li_Azz6ewULS8wFn1SYu,item_level_coupon,700.0,MIGRA2024STARTER1USER,MIGRA2024STARTER1USER,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
65,95371,44756,6538aee581e02900329122e6,186661,Mexico,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 23:07:08-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,95371,44756,6538aee581e02900329122e6,186662,Mexico,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 23:11:58-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
67,95371,44756,6538aee581e02900329122e6,186663,Mexico,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 23:20:18-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
68,95371,44756,6538aee581e02900329122e6,186664,Mexico,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 23:28:43-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
69,95371,44756,6538aee581e02900329122e6,186665,Mexico,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-14 23:45:33-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*Paquete de HSM 16000 USD 978/ ...,customer,6538aee581e02900329122e6
70,95371,44756,6538aee581e02900329122e6,186667,Mexico,AzZKRtTuUFPtK4J4j,True,paid,tax_exclusive,2024-08-15 00:09:58-03:00,...,NaN,line_item_discount,li_16BdC4ULSSprV83UH,item_level_coupon,700.0,MIGRA24A23STARTER1AGENTEX3MESES,MIGRA24A23STARTER1AGENTEX3MESES,NaN,NaN,NaN


In [330]:
ono[['total', 'quantity', 'amount']]

,total,quantity,amount
61,16466,400,13000
62,686100,6,0
63,606,2,1000
64,606,400,13000
65,605,1,3900
66,605,2,700
67,605,1,3900
68,605,1,0
69,605,50,2400
70,604,1,8900
